<a href="https://colab.research.google.com/github/korkutanapa/ANOMALY_DETECTION_TDA_YAHOO_DATASET/blob/main/TDA_CODES_for_NAB_ORIGINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VERSION 1

In [ ]:
# @title
import os
import shutil
import glob
import numpy as np
import pandas as pd

# ============================================================
# 1. CLEAN START & CLONE NAB
# ============================================================
print("--- 1. CLEAN START ---")
os.chdir("/content")

# Remove old NAB clone if exists
if os.path.exists("NAB"):
    shutil.rmtree("NAB")

# Clone NAB repository
!git clone https://github.com/numenta/NAB.git

# Install ripser for TDA
!pip install -q ripser

os.chdir("/content/NAB")

# Ensure config folder + empty thresholds.json (optimize will fill it)
os.makedirs("config", exist_ok=True)
thr_path = os.path.join("config", "thresholds.json")
if not os.path.exists(thr_path):
    with open(thr_path, "w") as f:
        f.write("{}")


# ============================================================
# 3. WRITE TDA_VEAD_METHOD (my_algo.py)
# ============================================================
print("--- 3. WRITING TDA_VEAD_METHOD DETECTOR ---")

tda_code = """
import os
import glob
import numpy as np
import pandas as pd
from ripser import ripser
import warnings

warnings.filterwarnings("ignore")

DETECTOR_NAME = "TDA_VEAD_Method"
INPUT_DIR = "data"
OUTPUT_DIR = os.path.join("results", DETECTOR_NAME)

# ----------------------------------------------------------
# Embedding parameters (fixed in NAB for fairness)
# ----------------------------------------------------------
WINDOW_SIZE = 14   # sliding window length for raw time series
TAU         = 1
DIMENSION   = 7    # must satisfy (DIMENSION-1)*TAU < WINDOW_SIZE
_EPS = 1e-12

# ==========================================================
# 0. VEAD CONFIGURATION (YOUR ORIGINAL SETTINGS)
# ==========================================================
KV   = 3.5          # Velocity gain
KA   = 3.5          # Acceleration gain
MODE = "abs_plateau"  # "strict" | "plateau" | "abs_plateau"

def _vead_series(raw_vals, kv=KV, ka=KA, mode=MODE):
    \"""
    VEAD score for a 1D series (raw_vals).

    Steps:
      1. Interpolate NaNs.
      2. Compute velocity v = diff(x).
      3. Compute acceleration a = diff(v).
      4. Robust Z-score with MAD for v and a.
      5. Apply mode logic (strict / plateau / abs_plateau).
      6. Final score = (kv * zv) * (ka * za).
    \"""
    # 1) Convert to pandas Series & interpolate
    s = pd.to_numeric(pd.Series(raw_vals, dtype=float), errors="coerce") \
            .interpolate(limit_direction="both")

    # 2) Velocity and acceleration
    v = s.diff(1)
    a = v.diff(1)

    # 3) Robust Z-score with MAD
    def _zmad(x):
        x = np.asarray(x, dtype=float)
        med = np.nanmedian(x)
        mad = np.nanmedian(np.abs(x - med)) + 1e-12
        return (x - med) / mad

    zv = _zmad(v.values)
    za = _zmad(a.values)

    # 4) Mode logic
    mode = (mode or "strict").lower()
    if mode == "strict":
        # Only positive shocks
        zv = np.maximum(0.0, zv)
        za = np.maximum(0.0, za)
    elif mode == "plateau":
        # Allow small negative dips, silence large negative drops
        zv = np.where(zv > -0.25, zv, 0.0)
        za = np.where(za > -0.25, za, 0.0)
    elif mode == "abs_plateau":
        # Absolute shock magnitude (spikes + drops)
        zv = np.abs(zv)
        za = np.abs(za)

    # 5) VEAD energy
    score = (kv * zv) * (ka * za)

    # 6) Clean NaNs / infs
    return np.nan_to_num(score, nan=0.0, posinf=0.0, neginf=0.0)

# ==========================================================
# 1. TDA FEATURE NAMES (H0-BASED)
# ==========================================================
FEATURE_NAMES = [
    "H0_ratio_auc_L1_to_sum",
    "H0_ratio_auc_to_max",
    "H0_ratio_auc_to_l2",
    "H0_bottleneck",
    "tail_share_q90",
    "H0_sum_centroid",
    "H0_L2_norm",
    "PETE_p1.6_q0.5",
    "H0_energy_concentration",
    "H0_dominance_share",
    "H0_tail_curvature_80_90",
    "H0_centroid_to_energy",
    "H0_gini",
]


# ==========================================================
# 2. TDA UTILITY FUNCTIONS (ADAPTED FROM YOUR CODE)
# ==========================================================
def takens_embed(window: np.ndarray, tau: int, m: int) -> np.ndarray:
    \"""
    1D Takens embedding for a given window:
      window: 1D array length N
      tau: delay
      m: embedding dimension
    Returns shape (m, L) where L = N - (m-1)*tau, or None if not enough points.
    \"""
    L = len(window) - (m - 1) * tau
    if L <= 0:
        return None
    return np.stack([window[j : j + L * tau : tau] for j in range(m)], axis=1)


def _clean_diag_h(diag_h):
    if diag_h is None:
        return np.empty((0, 2), dtype=float)
    arr = np.asarray(diag_h, dtype=float)
    if arr.ndim != 2 or arr.shape[1] != 2 or arr.size == 0:
        return np.empty((0, 2), dtype=float)
    finite_mask = np.isfinite(arr).all(axis=1)
    arr = arr[finite_mask]
    if arr.size == 0:
        return np.empty((0, 2), dtype=float)
    b, d = arr[:, 0], arr[:, 1]
    ok = np.isfinite(d) & (d > b)
    if not np.any(ok):
        return np.empty((0, 2), dtype=float)
    return np.stack([b[ok], d[ok]], axis=1)


try:
    _trapz = np.trapezoid
except AttributeError:
    _trapz = np.trapz


def _lifetimes(arr):
    return np.maximum(arr[:, 1] - arr[:, 0], 0.0) if arr.size else np.empty((0,), float)


def _bottleneck_amp(arr):
    L = _lifetimes(arr)
    return float(np.max(L)) if L.size else 0.0


def h0_l2_norm(arr):
    L = _lifetimes(arr)
    return float(np.sqrt(np.sum(L**2))) if L.size else 0.0


def _auc_tri_max(arr):
    if arr.size == 0:
        return 0.0
    if arr.shape[0] == 1:
        return 0.25 * ((arr[0, 1] - arr[0, 0])**2)

    # Simple grid approximation for AUC
    n_grid = 64
    lo, hi = (float(np.min(arr[:, 0])), float(np.max(arr[:, 1]))) if arr.size else (0.0, 1.0)
    grid = np.linspace(lo, hi, num=n_grid)
    lam1 = np.zeros_like(grid, float)

    b, d = arr[:, 0], arr[:, 1]
    for bj, dj in zip(b, d):
        m = 0.5 * (bj + dj)
        h = 0.5 * (dj - bj)
        if h <= 0:
            continue

        mask = (grid >= bj) & (grid <= dj)
        if not mask.any():
            continue

        # Left side of triangle
        l_mask = mask & (grid <= m)
        if l_mask.any():
            lam1[l_mask] = np.maximum(lam1[l_mask], (grid[l_mask] - bj) * (h / (m - bj + _EPS)))

        # Right side of triangle
        r_mask = mask & (grid > m)
        if r_mask.any():
            lam1[r_mask] = np.maximum(lam1[r_mask], (dj - grid[r_mask]) * (h / (dj - m + _EPS)))

    return float(_trapz(lam1, grid))


def compute_h0_features_for_window(window: np.ndarray) -> dict:
    \"""
    Compute all H0-based TDA features for one window.
    Returns a dict with keys in FEATURE_NAMES.
    \"""
    try:
        emb = takens_embed(window, TAU, DIMENSION)
        if emb is None:
            # Not enough points
            return {name: 0.0 for name in FEATURE_NAMES}

        dgms = ripser(emb, maxdim=0)["dgms"]
        D0 = _clean_diag_h(dgms[0] if len(dgms) else None)

        L = _lifetimes(D0)
        S = float(L.sum())
        A = _auc_tri_max(D0)
        L2 = h0_l2_norm(D0)
        mx = float(np.max(L)) if L.size else 0.0

        feats = {}

        # 1) Ratios
        feats["H0_ratio_auc_L1_to_sum"] = 0.0 if S <= _EPS else A / S
        feats["H0_ratio_auc_to_max"] = 0.0 if mx <= _EPS else A / mx
        feats["H0_ratio_auc_to_l2"] = 0.0 if L2 <= _EPS else A / L2

        # 2) Bottleneck
        feats["H0_bottleneck"] = mx

        # 3) Tail share q90
        if S > _EPS and L.size > 0:
            qv = float(np.quantile(L, 0.90))
            feats["tail_share_q90"] = float(L[L >= qv].sum()) / S
        else:
            feats["tail_share_q90"] = 0.0

        # 4) Centroid
        if S > _EPS and D0.size > 0:
            radial = np.abs((D0[:, 0] + D0[:, 1]) / np.sqrt(2.0))
            feats["H0_sum_centroid"] = float(np.sum(radial * L)) / S
        else:
            feats["H0_sum_centroid"] = 0.0

        # 5) L2 norm
        feats["H0_L2_norm"] = L2

        # 6) PETE
        if S > _EPS and D0.size > 0:
            radial = (D0[:, 0] + D0[:, 1]) / np.sqrt(2.0)
            num = np.sum((L ** 1.6) * (np.abs(radial) ** 0.5))
            feats["PETE_p1.6_q0.5"] = num / S
        else:
            feats["PETE_p1.6_q0.5"] = 0.0

        # 7) Energy concentration & dominance
        feats["H0_energy_concentration"] = (L2 / S) if S > _EPS else 0.0
        feats["H0_dominance_share"] = (mx / S) if S > _EPS else 0.0

        # 8) Tail curvature
        if S > _EPS and L.size > 0:
            q90 = float(L[L >= np.quantile(L, 0.90)].sum()) / S
            q80 = float(L[L >= np.quantile(L, 0.80)].sum()) / S
            feats["H0_tail_curvature_80_90"] = q90 - q80
        else:
            feats["H0_tail_curvature_80_90"] = 0.0

        # 9) Centroid to energy
        feats["H0_centroid_to_energy"] = (
            feats["H0_sum_centroid"] / L2
        ) if L2 > _EPS else 0.0

        # 10) Gini
        if L.size > 0 and S > _EPS:
            xs = np.sort(L)
            n = xs.size
            cumx = np.cumsum(xs)
            feats["H0_gini"] = float(1.0 + 1.0/n - 2.0 * (cumx.sum() / (n * S)))
        else:
            feats["H0_gini"] = 0.0

        # Ensure all feature names are present
        for name in FEATURE_NAMES:
            feats.setdefault(name, 0.0)

        return feats

    except Exception:
        # On any failure, return zeros for all features
        return {name: 0.0 for name in FEATURE_NAMES}


# ==========================================================
# 3. MAIN: ASK USER FOR FEATURE, THEN RUN NAB DETECTOR
# ==========================================================
def run():
    # --- Ask user to pick a TDA feature ---
    print("\\nAvailable TDA H0 features:")
    for idx, name in enumerate(FEATURE_NAMES):
        print(f"  {idx:2d} -> {name}")
    choice = input("Select feature by index or name (default: H0_bottleneck): ").strip()

    selected_feature = "H0_bottleneck"

    if choice == "":
        pass
    elif choice.isdigit():
        idx = int(choice)
        if 0 <= idx < len(FEATURE_NAMES):
            selected_feature = FEATURE_NAMES[idx]
        else:
            print(f"Index {idx} out of range, using default H0_bottleneck.")
    else:
        if choice in FEATURE_NAMES:
            selected_feature = choice
        else:
            print(f"Feature '{choice}' not recognized, using default H0_bottleneck.")

    print(f"\\n>>> Using TDA feature: {selected_feature}")
    print(f">>> VEAD mode   : {MODE}")
    print(f">>> KV, KA       : {KV}, {KA}\\n")

    # --- Process all NAB CSV files ---
    files = glob.glob(os.path.join(INPUT_DIR, "**", "*.csv"), recursive=True)
    print(f"   Found {len(files)} data files in '{INPUT_DIR}'")

    for filepath in files:
        if ".ipynb_checkpoints" in filepath:
            continue

        try:
            df = pd.read_csv(filepath)
            df.columns = [c.strip().lower() for c in df.columns]

            if "value" not in df.columns or "timestamp" not in df.columns:
                continue

            vals = df["value"].astype(float).values

            # 1) Compute chosen TDA feature over sliding windows
            scores = []
            for i in range(len(vals)):
                if i < WINDOW_SIZE:
                    scores.append(0.0)
                else:
                    window = vals[i - WINDOW_SIZE : i]
                    feats = compute_h0_features_for_window(window)
                    scores.append(float(feats.get(selected_feature, 0.0)))

            scores = pd.Series(scores, dtype=float).fillna(0.0)

            # 2) Apply full VEAD algorithm to the chosen TDA feature
            vead_scores = _vead_series(scores.values, kv=KV, ka=KA, mode=MODE)
            vead_series = pd.Series(vead_scores, index=df.index)

            # 3) Normalize VEAD scores into [0, 1] for NAB
            max_val = vead_series.max()
            if pd.isna(max_val) or max_val <= 0:
                final_scores = vead_series * 0.0
            else:
                final_scores = (vead_series / max_val).clip(lower=0.0, upper=1.0)

            # 4) Build output path: results/TDA_VEAD_Method/<category>/TDA_VEAD_Method_<file>.csv
            rel = os.path.relpath(filepath, INPUT_DIR)
            category = os.path.dirname(rel)
            base_name = os.path.basename(rel)

            out_dir = os.path.join(OUTPUT_DIR, category)
            os.makedirs(out_dir, exist_ok=True)
            out_name = f"{DETECTOR_NAME}_" + base_name
            out_path = os.path.join(out_dir, out_name)

            # 5) Write result: timestamp + anomaly_score
            out_df = pd.DataFrame({
                "timestamp": df["timestamp"],
                "anomaly_score": final_scores.values
            })
            out_df.to_csv(out_path, index=False)
            print(f"   -> Wrote: {out_path}")

        except Exception as e:
            print(f"   !! Error processing {filepath}: {e}")
            continue


if __name__ == "__main__":
    run()
"""
with open("my_algo.py", "w") as f:
    f.write(tda_code)

print("✅ my_algo.py written.")

# ============================================================
# 4. RUN YOUR DETECTOR ON ALL NAB DATA
# ============================================================
print("--- 4. RUNNING TDA_VEAD_Method ON ALL DATASETS ---")
!python my_algo.py

# ============================================================
# 5. RUN NAB OPTIMIZE + SCORE FOR THIS DETECTOR
# ============================================================
print("--- 5. RUNNING NAB OPTIMIZE + SCORE ---")
!python run.py --optimize --score --detectors TDA_VEAD_Method --normalize


--- 1. CLEAN START ---
Cloning into 'NAB'...
remote: Enumerating objects: 7119, done.
remote: Counting objects: 100% (699/699), done.
remote: Compressing objects: 100% (204/204), done.
remote: Total 7119 (delta 552), reused 495 (delta 495), pack-reused 6420 (from 1)
Receiving objects: 100% (7119/7119), 86.13 MiB | 22.46 MiB/s, done.
Resolving deltas: 100% (5001/5001), done.
Updating files: 100% (1186/1186), done.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 842.1/842.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 3.7 MB/s eta 0:00:00
--- 3. WRITING TDA_VEAD_METHOD DETECTOR ---
✅ my_algo.py written.
--- 4. RUNNING TDA_VEAD_Method ON ALL DATASETS ---

Available TDA H0 features:
   0 -> H0_ratio_auc_L1_to_sum
   1 -> H0_ratio_auc_to_max
   2 -> H0_ratio_auc_to_l2
   3 -> H0_bottleneck
   4 -> tail_share_q90
   5 -> H0_sum_centroid
   6 -> H0_L2_norm
   7 -> PETE_p1.6_q0.5
   8 -> H0_energy_concentration
   9 -> H

# Version 2 No VEAD

In [ ]:
# @title
import os
import shutil
import glob
import numpy as np
import pandas as pd

# ============================================================
# 1. CLEAN START & CLONE NAB
# ============================================================
print("--- 1. CLEAN START ---")
os.chdir("/content")

# Remove old NAB clone if exists
if os.path.exists("NAB"):
    shutil.rmtree("NAB")

# Clone NAB repository
!git clone https://github.com/numenta/NAB.git

# Install ripser for TDA
!pip install -q ripser

os.chdir("/content/NAB")

# Ensure config folder + empty thresholds.json (optimize will fill it)
os.makedirs("config", exist_ok=True)
thr_path = os.path.join("config", "thresholds.json")
if not os.path.exists(thr_path):
    with open(thr_path, "w") as f:
        f.write("{}")

# ============================================================
# 3. WRITE TDA_METHOD (my_algo.py)  -- NO VEAD
# ============================================================
print("--- 3. WRITING TDA_METHOD DETECTOR (NO VEAD) ---")

tda_code = r"""
import os
import glob
import numpy as np
import pandas as pd
from ripser import ripser
import warnings

warnings.filterwarnings("ignore")

DETECTOR_NAME = "TDA_Method_NoVEAD"
INPUT_DIR = "data"
OUTPUT_DIR = os.path.join("results", DETECTOR_NAME)

# ----------------------------------------------------------
# Embedding parameters (fixed in NAB for fairness)
# ----------------------------------------------------------
WINDOW_SIZE = 14
TAU         = 1
DIMENSION   = 7    # must satisfy (DIMENSION-1)*TAU < WINDOW_SIZE
_EPS = 1e-12

# ==========================================================
# 1. TDA FEATURE NAMES (H0-BASED)
# ==========================================================
FEATURE_NAMES = [
    "H0_ratio_auc_L1_to_sum",
    "H0_ratio_auc_to_max",
    "H0_ratio_auc_to_l2",
    "H0_bottleneck",
    "tail_share_q90",
    "H0_sum_centroid",
    "H0_L2_norm",
    "PETE_p1.6_q0.5",
    "H0_energy_concentration",
    "H0_dominance_share",
    "H0_tail_curvature_80_90",
    "H0_centroid_to_energy",
    "H0_gini",
]

# ==========================================================
# 2. TDA UTILITY FUNCTIONS
# ==========================================================
def takens_embed(window: np.ndarray, tau: int, m: int) -> np.ndarray:
    L = len(window) - (m - 1) * tau
    if L <= 0:
        return None
    return np.stack([window[j : j + L * tau : tau] for j in range(m)], axis=1)

def _clean_diag_h(diag_h):
    if diag_h is None:
        return np.empty((0, 2), dtype=float)
    arr = np.asarray(diag_h, dtype=float)
    if arr.ndim != 2 or arr.shape[1] != 2 or arr.size == 0:
        return np.empty((0, 2), dtype=float)
    finite_mask = np.isfinite(arr).all(axis=1)
    arr = arr[finite_mask]
    if arr.size == 0:
        return np.empty((0, 2), dtype=float)
    b, d = arr[:, 0], arr[:, 1]
    ok = np.isfinite(d) & (d > b)
    if not np.any(ok):
        return np.empty((0, 2), dtype=float)
    return np.stack([b[ok], d[ok]], axis=1)

try:
    _trapz = np.trapezoid
except AttributeError:
    _trapz = np.trapz

def _lifetimes(arr):
    return np.maximum(arr[:, 1] - arr[:, 0], 0.0) if arr.size else np.empty((0,), float)

def h0_l2_norm(arr):
    L = _lifetimes(arr)
    return float(np.sqrt(np.sum(L**2))) if L.size else 0.0

def _auc_tri_max(arr):
    if arr.size == 0:
        return 0.0
    if arr.shape[0] == 1:
        return 0.25 * ((arr[0, 1] - arr[0, 0])**2)

    n_grid = 64
    lo, hi = (float(np.min(arr[:, 0])), float(np.max(arr[:, 1])))
    grid = np.linspace(lo, hi, num=n_grid)
    lam1 = np.zeros_like(grid, float)

    b, d = arr[:, 0], arr[:, 1]
    for bj, dj in zip(b, d):
        m = 0.5 * (bj + dj)
        h = 0.5 * (dj - bj)
        if h <= 0:
            continue

        mask = (grid >= bj) & (grid <= dj)
        if not mask.any():
            continue

        l_mask = mask & (grid <= m)
        if l_mask.any():
            lam1[l_mask] = np.maximum(lam1[l_mask], (grid[l_mask] - bj) * (h / (m - bj + _EPS)))

        r_mask = mask & (grid > m)
        if r_mask.any():
            lam1[r_mask] = np.maximum(lam1[r_mask], (dj - grid[r_mask]) * (h / (dj - m + _EPS)))

    return float(_trapz(lam1, grid))

def compute_h0_features_for_window(window: np.ndarray) -> dict:
    try:
        emb = takens_embed(window, TAU, DIMENSION)
        if emb is None:
            return {name: 0.0 for name in FEATURE_NAMES}

        dgms = ripser(emb, maxdim=0)["dgms"]
        D0 = _clean_diag_h(dgms[0] if len(dgms) else None)

        L = _lifetimes(D0)
        S = float(L.sum())
        A = _auc_tri_max(D0)
        L2 = h0_l2_norm(D0)
        mx = float(np.max(L)) if L.size else 0.0

        feats = {}
        feats["H0_ratio_auc_L1_to_sum"] = 0.0 if S <= _EPS else A / S
        feats["H0_ratio_auc_to_max"]    = 0.0 if mx <= _EPS else A / mx
        feats["H0_ratio_auc_to_l2"]     = 0.0 if L2 <= _EPS else A / L2
        feats["H0_bottleneck"]          = mx

        if S > _EPS and L.size > 0:
            qv = float(np.quantile(L, 0.90))
            feats["tail_share_q90"] = float(L[L >= qv].sum()) / S
        else:
            feats["tail_share_q90"] = 0.0

        if S > _EPS and D0.size > 0:
            radial = np.abs((D0[:, 0] + D0[:, 1]) / np.sqrt(2.0))
            feats["H0_sum_centroid"] = float(np.sum(radial * L)) / S
        else:
            feats["H0_sum_centroid"] = 0.0

        feats["H0_L2_norm"] = L2

        if S > _EPS and D0.size > 0:
            radial = (D0[:, 0] + D0[:, 1]) / np.sqrt(2.0)
            num = np.sum((L ** 1.6) * (np.abs(radial) ** 0.5))
            feats["PETE_p1.6_q0.5"] = num / S
        else:
            feats["PETE_p1.6_q0.5"] = 0.0

        feats["H0_energy_concentration"] = (L2 / S) if S > _EPS else 0.0
        feats["H0_dominance_share"]      = (mx / S) if S > _EPS else 0.0

        if S > _EPS and L.size > 0:
            q90 = float(L[L >= np.quantile(L, 0.90)].sum()) / S
            q80 = float(L[L >= np.quantile(L, 0.80)].sum()) / S
            feats["H0_tail_curvature_80_90"] = q90 - q80
        else:
            feats["H0_tail_curvature_80_90"] = 0.0

        feats["H0_centroid_to_energy"] = (feats["H0_sum_centroid"] / L2) if L2 > _EPS else 0.0

        if L.size > 0 and S > _EPS:
            xs = np.sort(L)
            n = xs.size
            cumx = np.cumsum(xs)
            feats["H0_gini"] = float(1.0 + 1.0/n - 2.0 * (cumx.sum() / (n * S)))
        else:
            feats["H0_gini"] = 0.0

        for name in FEATURE_NAMES:
            feats.setdefault(name, 0.0)

        return feats
    except Exception:
        return {name: 0.0 for name in FEATURE_NAMES}

def _minmax_01(x: np.ndarray) -> np.ndarray:
    x = np.asarray(x, dtype=float)
    x = np.nan_to_num(x, nan=0.0, posinf=0.0, neginf=0.0)
    mn = float(np.min(x)) if x.size else 0.0
    mx = float(np.max(x)) if x.size else 0.0
    if (mx - mn) <= _EPS:
        return np.zeros_like(x, dtype=float)
    return (x - mn) / (mx - mn)

# ==========================================================
# 3. MAIN: ASK USER FOR FEATURE, THEN RUN NAB DETECTOR
# ==========================================================
def run():
    print("\nAvailable TDA H0 features:")
    for idx, name in enumerate(FEATURE_NAMES):
        print(f"  {idx:2d} -> {name}")
    choice = input("Select feature by index or name (default: H0_bottleneck): ").strip()

    selected_feature = "H0_bottleneck"
    if choice == "":
        pass
    elif choice.isdigit():
        idx = int(choice)
        if 0 <= idx < len(FEATURE_NAMES):
            selected_feature = FEATURE_NAMES[idx]
        else:
            print(f"Index {idx} out of range, using default H0_bottleneck.")
    else:
        if choice in FEATURE_NAMES:
            selected_feature = choice
        else:
            print(f"Feature '{choice}' not recognized, using default H0_bottleneck.")

    print(f"\n>>> Using TDA feature (NO VEAD): {selected_feature}\n")

    files = glob.glob(os.path.join(INPUT_DIR, "**", "*.csv"), recursive=True)
    print(f"   Found {len(files)} data files in '{INPUT_DIR}'")

    for filepath in files:
        if ".ipynb_checkpoints" in filepath:
            continue

        try:
            df = pd.read_csv(filepath)
            df.columns = [c.strip().lower() for c in df.columns]

            if "value" not in df.columns or "timestamp" not in df.columns:
                continue

            vals = df["value"].astype(float).values

            # 1) Compute chosen TDA feature over sliding windows
            feat_series = np.zeros(len(vals), dtype=float)
            for i in range(len(vals)):
                if i < WINDOW_SIZE:
                    feat_series[i] = 0.0
                else:
                    window = vals[i - WINDOW_SIZE : i]
                    feats = compute_h0_features_for_window(window)
                    feat_series[i] = float(feats.get(selected_feature, 0.0))

            # 2) Normalize feature directly into [0, 1] for NAB
            final_scores = _minmax_01(feat_series)

            # 3) Build output path
            rel = os.path.relpath(filepath, INPUT_DIR)
            category = os.path.dirname(rel)
            base_name = os.path.basename(rel)

            out_dir = os.path.join(OUTPUT_DIR, category)
            os.makedirs(out_dir, exist_ok=True)
            out_name = f"{DETECTOR_NAME}_" + base_name
            out_path = os.path.join(out_dir, out_name)

            # 4) Write result: timestamp + anomaly_score
            out_df = pd.DataFrame({
                "timestamp": df["timestamp"],
                "anomaly_score": final_scores
            })
            out_df.to_csv(out_path, index=False)
            print(f"   -> Wrote: {out_path}")

        except Exception as e:
            print(f"   !! Error processing {filepath}: {e}")
            continue

if __name__ == "__main__":
    run()
"""

with open("my_algo.py", "w") as f:
    f.write(tda_code)

print("✅ my_algo.py written.")

# ============================================================
# 4. RUN YOUR DETECTOR ON ALL NAB DATA
# ============================================================
print("--- 4. RUNNING TDA_Method_NoVEAD ON ALL DATASETS ---")
!python my_algo.py

# ============================================================
# 5. RUN NAB OPTIMIZE + SCORE FOR THIS DETECTOR
# ============================================================
print("--- 5. RUNNING NAB OPTIMIZE + SCORE ---")
!python run.py --optimize --score --detectors TDA_Method_NoVEAD --normalize


--- 1. CLEAN START ---
Cloning into 'NAB'...
remote: Enumerating objects: 7119, done.
remote: Counting objects: 100% (699/699), done.
remote: Compressing objects: 100% (204/204), done.
remote: Total 7119 (delta 552), reused 495 (delta 495), pack-reused 6420 (from 1)
Receiving objects: 100% (7119/7119), 86.13 MiB | 23.23 MiB/s, done.
Resolving deltas: 100% (5001/5001), done.
Updating files: 100% (1186/1186), done.
--- 3. WRITING TDA_METHOD DETECTOR (NO VEAD) ---
✅ my_algo.py written.
--- 4. RUNNING TDA_Method_NoVEAD ON ALL DATASETS ---

Available TDA H0 features:
   0 -> H0_ratio_auc_L1_to_sum
   1 -> H0_ratio_auc_to_max
   2 -> H0_ratio_auc_to_l2
   3 -> H0_bottleneck
   4 -> tail_share_q90
   5 -> H0_sum_centroid
   6 -> H0_L2_norm
   7 -> PETE_p1.6_q0.5
   8 -> H0_energy_concentration
   9 -> H0_dominance_share
  10 -> H0_tail_curvature_80_90
  11 -> H0_centroid_to_energy
  12 -> H0_gini
Select feature by index or name (default: H0_bottleneck): 7

>>> Using TDA feature (NO VEAD): PET

# Version 3 full flow w 77 FEATURES

Takens embedding: turns the 1D window into a point cloud (delay embedding).

ripser: computes persistence diagrams H0 and H1 from that point cloud.

Feature extraction: computes many statistics from diagrams (lifetimes, entropy, gini, Carlsson, etc.).

Feature selection: you pick one of the 77 features.

VEAD: treats that chosen feature as a time series, computes velocity + acceleration shocks with robust MAD Z-scores, then multiplies them.

Normalize: required by NAB to output scores in [0,1].

Write results: NAB expects results/<detector>/.../*.csv with timestamp, anomaly_score.

run.py optimize/score: NAB finds thresholds and computes official NAB scores.

If you want, I can also generate a single combined flowchart (Notebook + Detector + NAB scoring) in one diagram, or a very compact 8-box version for slides.

In [ ]:
# @title
import os
import shutil
import glob
import numpy as np
import pandas as pd

# ============================================================
# 1. CLEAN START & CLONE NAB
# ============================================================
print("--- 1. CLEAN START ---")
os.chdir("/content")

if os.path.exists("NAB"):
    shutil.rmtree("NAB")

!git clone https://github.com/numenta/NAB.git
!pip install -q ripser

os.chdir("/content/NAB")

os.makedirs("config", exist_ok=True)
thr_path = os.path.join("config", "thresholds.json")
if not os.path.exists(thr_path):
    with open(thr_path, "w") as f:
        f.write("{}")

# ============================================================
# 3. WRITE TDA_VEAD_METHOD (my_algo.py)  [VEAD + 77 FEATURES]
# ============================================================
print("--- 3. WRITING TDA_VEAD_METHOD DETECTOR (VEAD + 77 FEATURES) ---")

tda_code = r"""
import os
import glob
import numpy as np
import pandas as pd
from ripser import ripser
import warnings

warnings.filterwarnings("ignore")

DETECTOR_NAME = "TDA_VEAD_Method"
INPUT_DIR = "data"
OUTPUT_DIR = os.path.join("results", DETECTOR_NAME)

# ----------------------------------------------------------
# Embedding parameters (fixed in NAB for fairness)
# ----------------------------------------------------------
WINDOW_SIZE = 14
TAU         = 1
DIMENSION   = 7
_EPS        = 1e-12

MAXDIM = 1  # H0 + H1

# ==========================================================
# 0. VEAD CONFIGURATION (UNCHANGED)
# ==========================================================
KV   = 3.5
KA   = 3.5
MODE = "abs_plateau"  # "strict" | "plateau" | "abs_plateau"

def _vead_series(raw_vals, kv=KV, ka=KA, mode=MODE):
    # 1) Convert to pandas Series & interpolate
    s = pd.to_numeric(pd.Series(raw_vals, dtype=float), errors="coerce") \
            .interpolate(limit_direction="both")

    # 2) Velocity and acceleration
    v = s.diff(1)
    a = v.diff(1)

    # 3) Robust Z-score with MAD
    def _zmad(x):
        x = np.asarray(x, dtype=float)
        med = np.nanmedian(x)
        mad = np.nanmedian(np.abs(x - med)) + 1e-12
        return (x - med) / mad

    zv = _zmad(v.values)
    za = _zmad(a.values)

    # 4) Mode logic
    mode = (mode or "strict").lower()
    if mode == "strict":
        zv = np.maximum(0.0, zv)
        za = np.maximum(0.0, za)
    elif mode == "plateau":
        zv = np.where(zv > -0.25, zv, 0.0)
        za = np.where(za > -0.25, za, 0.0)
    elif mode == "abs_plateau":
        zv = np.abs(zv)
        za = np.abs(za)

    score = (kv * zv) * (ka * za)
    return np.nan_to_num(score, nan=0.0, posinf=0.0, neginf=0.0)

# ==========================================================
# 1. TAKENS EMBEDDING
# ==========================================================
def takens_embed(window, time_delay, dimension):
    m = len(window) - (dimension - 1) * time_delay
    if m <= 0:
        raise ValueError("Takens parameters too large for this window.")
    return np.stack(
        [window[j:j + m * time_delay:time_delay] for j in range(dimension)],
        axis=1
    )

# ==========================================================
# 2. PERSISTENCE DIAGRAM UTILITIES + FEATURE FUNCTIONS
#    (COPIED/ALIGNED WITH YOUR 77-FEATURE CODE)
# ==========================================================
def _clean_diag(diag):
    if diag is None:
        return np.empty((0, 2), dtype=float)
    arr = np.asarray(diag, dtype=float)
    if arr.ndim != 2 or arr.shape[1] != 2 or arr.size == 0:
        return np.empty((0, 2), dtype=float)
    b, d = arr[:, 0], arr[:, 1]
    mask = np.isfinite(b) & np.isfinite(d) & (d > b)
    if not np.any(mask):
        return np.empty((0, 2), dtype=float)
    return np.stack([b[mask], d[mask]], axis=1)

def _lifetimes(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return np.empty(0, dtype=float)
    return np.maximum(arr[:, 1] - arr[:, 0], 0.0)

def _safe_div(a, b):
    return float(a) / float(b + _EPS)

try:
    _trapz = np.trapezoid
except AttributeError:
    _trapz = np.trapz

def _auc_tri_max(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return 0.0
    b_all, d_all = arr[:, 0], arr[:, 1]
    if b_all.min() == d_all.max():
        return 0.0

    grid = np.linspace(b_all.min(), d_all.max(), 64)
    lam1 = np.zeros_like(grid)

    for b, d in arr:
        m = 0.5 * (b + d)
        h = 0.5 * (d - b)
        if h <= 0:
            continue

        left = (grid >= b) & (grid <= m)
        right = (grid >= m) & (grid <= d)

        lam1[left] = np.maximum(lam1[left], (grid[left] - b) * (h / max(m - b, _EPS)))
        lam1[right] = np.maximum(lam1[right], (d - grid[right]) * (h / max(d - m, _EPS)))

    return float(_trapz(lam1, grid))

def _persistence_entropy(diag):
    L = _lifetimes(diag)
    if L.size == 0:
        return 0.0
    S = L.sum()
    if S <= 0:
        return 0.0
    p = L / (S + _EPS)
    return float(-np.sum(p * np.log(p + _EPS)))

def _gini_from_lifetimes(L):
    L = np.sort(L)
    n = len(L)
    if n == 0:
        return 0.0
    S = L.sum()
    if S <= 0:
        return 0.0
    cumL = np.cumsum(L)
    return float(1 + 1/n - 2*np.sum(cumL/(n*S)))

def _tail_share_q(diag, q):
    L = _lifetimes(diag)
    if L.size == 0:
        return 0.0
    qv = np.quantile(L, q)
    return _safe_div(L[L >= qv].sum(), L.sum())

def _birth_death_stats(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return {"mean_birth": 0.0, "mean_death": 0.0, "std_birth": 0.0, "std_death": 0.0}
    b, d = arr[:, 0], arr[:, 1]
    return {
        "mean_birth": float(b.mean()),
        "mean_death": float(d.mean()),
        "std_birth": float(b.std(ddof=0)),
        "std_death": float(d.std(ddof=0)),
    }

def _diag_distance_stats(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return {"mean_diag_dist": 0.0, "max_diag_dist": 0.0, "sum_diag_dist": 0.0}
    b, d = arr[:, 0], arr[:, 1]
    dist = (d - b) / np.sqrt(2.0)
    return {
        "mean_diag_dist": float(dist.mean()),
        "max_diag_dist": float(dist.max()),
        "sum_diag_dist": float(dist.sum()),
    }

def _centroid_xy(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return {"centroid_x": 0.0, "centroid_y": 0.0}
    b, d = arr[:, 0], arr[:, 1]
    L = np.maximum(d - b, 0.0)
    S = L.sum()
    if S <= 0:
        return {"centroid_x": 0.0, "centroid_y": 0.0}
    return {
        "centroid_x": float(np.sum(b * L) / (S + _EPS)),
        "centroid_y": float(np.sum(d * L) / (S + _EPS)),
    }

def _lifetimes_stats(diag):
    L = _lifetimes(diag)
    if L.size == 0:
        return {
            "count": 0, "sum": 0.0, "mean": 0.0, "median": 0.0, "std": 0.0,
            "min": 0.0, "max": 0.0, "L1": 0.0, "L2": 0.0, "Linf": 0.0
        }
    return {
        "count": int(L.size),
        "sum": float(L.sum()),
        "mean": float(L.mean()),
        "median": float(np.median(L)),
        "std": float(L.std(ddof=0)),
        "min": float(L.min()),
        "max": float(L.max()),
        "L1": float(np.sum(np.abs(L))),
        "L2": float(np.sqrt(np.sum(L**2))),
        "Linf": float(np.max(np.abs(L))),
    }

def _lifetimes_quantiles(diag):
    L = _lifetimes(diag)
    if L.size == 0:
        return {"q50": 0.0, "q75": 0.0, "q90": 0.0, "q95": 0.0, "q99": 0.0}
    return {
        "q50": float(np.quantile(L, 0.50)),
        "q75": float(np.quantile(L, 0.75)),
        "q90": float(np.quantile(L, 0.90)),
        "q95": float(np.quantile(L, 0.95)),
        "q99": float(np.quantile(L, 0.99)),
    }

def _carlsson_coordinates(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return {f"f{k}": 0.0 for k in range(1, 6)}
    b, d = arr[:, 0], arr[:, 1]
    L = np.maximum(d - b, 0.0)
    S = L.sum()
    if S <= 0:
        return {f"f{k}": 0.0 for k in range(1, 6)}
    return {
        "f1": float(L.sum()),
        "f2": float(np.sum(b * L)),
        "f3": float(np.sum(d * L)),
        "f4": float(np.sum(b**2 * L)),
        "f5": float(np.sum(d**2 * L)),
    }

def _sum_centroid_radial(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return 0.0
    b, d = arr[:, 0], arr[:, 1]
    L = np.maximum(d - b, 0.0)
    S = L.sum()
    if S <= 0:
        return 0.0
    radial = (b + d) / np.sqrt(2.0)
    return _safe_div(np.sum(np.abs(radial) * L), S)

def _pete(diag, p=1.6, q=0.5):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return 0.0
    b, d = arr[:, 0], arr[:, 1]
    L = np.maximum(d - b, 0.0)
    S = L.sum()
    if S <= 0:
        return 0.0
    radial = (b + d) / np.sqrt(2.0)
    return _safe_div(np.sum((L**p) * (np.abs(radial)**q)), S)

def compute_features_for_diag(diag, prefix):
    feats = {}

    Ls = _lifetimes_stats(diag)
    feats[f"{prefix}count_lifetime"] = float(Ls["count"])
    feats[f"{prefix}sum_lifetime"]   = float(Ls["sum"])
    feats[f"{prefix}mean_lifetime"]  = float(Ls["mean"])
    feats[f"{prefix}median_lifetime"]= float(Ls["median"])
    feats[f"{prefix}std_lifetime"]   = float(Ls["std"])
    feats[f"{prefix}min_lifetime"]   = float(Ls["min"])
    feats[f"{prefix}max_lifetime"]   = float(Ls["max"])

    feats[f"{prefix}L1_lifetime"]    = float(Ls["L1"])
    feats[f"{prefix}L2_lifetime"]    = float(Ls["L2"])
    feats[f"{prefix}Linf_lifetime"]  = float(Ls["Linf"])

    feats[f"{prefix}L1_norm"]        = float(Ls["L1"])
    feats[f"{prefix}L2_norm"]        = float(Ls["L2"])
    feats[f"{prefix}Linf_norm"]      = float(Ls["Linf"])

    feats[f"{prefix}betti"]          = float(Ls["count"])
    feats[f"{prefix}energy_concentration"] = _safe_div(Ls["L2"], Ls["L1"])
    feats[f"{prefix}dominance_share"]      = _safe_div(Ls["Linf"], Ls["L1"])

    feats[f"{prefix}persistence_entropy"]  = _persistence_entropy(diag)

    bd = _birth_death_stats(diag)
    for k, v in bd.items():
        feats[f"{prefix}{k}"] = float(v)

    dd = _diag_distance_stats(diag)
    for k, v in dd.items():
        feats[f"{prefix}{k}"] = float(v)

    cxy = _centroid_xy(diag)
    feats[f"{prefix}centroid_x"] = float(cxy["centroid_x"])
    feats[f"{prefix}centroid_y"] = float(cxy["centroid_y"])

    q = _lifetimes_quantiles(diag)
    for k, v in q.items():
        feats[f"{prefix}{k}"] = float(v)

    tail80 = _tail_share_q(diag, 0.80)
    tail90 = _tail_share_q(diag, 0.90)
    tail95 = _tail_share_q(diag, 0.95)

    feats[f"{prefix}tail_share_q80"] = float(tail80)
    feats[f"{prefix}tail_share_q90"] = float(tail90)
    feats[f"{prefix}tail_share_q95"] = float(tail95)
    feats[f"{prefix}tail_curvature_80_90"] = float(tail90 - tail80)

    L = _lifetimes(diag)
    feats[f"{prefix}gini"] = float(_gini_from_lifetimes(L))

    cc = _carlsson_coordinates(diag)
    feats[f"{prefix}Carlsson_f1"] = float(cc["f1"])
    feats[f"{prefix}Carlsson_f2"] = float(cc["f2"])
    feats[f"{prefix}Carlsson_f3"] = float(cc["f3"])
    feats[f"{prefix}Carlsson_f4"] = float(cc["f4"])
    feats[f"{prefix}Carlsson_f5"] = float(cc["f5"])

    if prefix == "H0_":
        A = _auc_tri_max(diag)
        feats["H0_ratio_auc_L1_to_sum"] = _safe_div(A, Ls["sum"])
        feats["H0_ratio_auc_to_max"]    = _safe_div(A, Ls["max"])
        feats["H0_ratio_auc_to_l2"]     = _safe_div(A, Ls["L2"])
        feats["H0_bottleneck"]          = float(Ls["max"])
        feats["H0_sum_centroid"]        = float(_sum_centroid_radial(diag))
        feats["PETE_p1.6_q0.5"]         = float(_pete(diag, p=1.6, q=0.5))
        feats["H0_energy_concentration"]= _safe_div(Ls["L2"], Ls["sum"])
        feats["H0_dominance_share"]     = _safe_div(Ls["Linf"], Ls["sum"])
        feats["H0_tail_curvature_80_90"]= float(tail90 - tail80)
        feats["H0_centroid_to_energy"]  = _safe_div(feats["H0_sum_centroid"], Ls["L2"])
        feats["H0_gini"]                = float(feats["H0_gini"])

    return feats

def compute_cross_dim_features(feats_H0, feats_H1):
    out = {}
    def g(d, k): return float(d.get(k, 0.0))
    out["H1_to_H0_betti_ratio"]   = _safe_div(g(feats_H1, "H1_betti"), g(feats_H0, "H0_betti"))
    out["H1_to_H0_entropy_ratio"] = _safe_div(g(feats_H1, "H1_persistence_entropy"), g(feats_H0, "H0_persistence_entropy"))
    return out

# ==========================================================
# 3. ROBUST FEATURES LIST (77)
# ==========================================================
ROBUST_FEATURES = [
    "H0_Carlsson_f1","H0_Carlsson_f3","H0_Carlsson_f5",
    "H0_L1_lifetime","H0_L1_norm","H0_L2_lifetime","H0_L2_norm",
    "H0_Linf_lifetime","H0_Linf_norm","H0_bottleneck","H0_centroid_to_energy",
    "H0_centroid_y","H0_dominance_share","H0_energy_concentration","H0_gini",
    "H0_max_diag_dist","H0_max_lifetime","H0_mean_death","H0_mean_diag_dist",
    "H0_mean_lifetime","H0_median_lifetime","H0_min_lifetime","H0_persistence_entropy",
    "H0_q50","H0_q75","H0_q90","H0_q95","H0_q99","H0_ratio_auc_L1_to_sum",
    "H0_ratio_auc_to_l2","H0_ratio_auc_to_max","H0_std_death","H0_std_lifetime",
    "H0_sum_centroid","H0_sum_diag_dist","H0_sum_lifetime","H0_tail_curvature_80_90",
    "H0_tail_share_q80","H0_tail_share_q90","H0_tail_share_q95",
    "H1_Carlsson_f1","H1_Carlsson_f2","H1_Carlsson_f3",
    "H1_L1_lifetime","H1_L1_norm","H1_L2_lifetime","H1_L2_norm",
    "H1_Linf_lifetime","H1_Linf_norm","H1_betti","H1_count_lifetime",
    "H1_dominance_share","H1_energy_concentration","H1_gini",
    "H1_max_diag_dist","H1_max_lifetime","H1_mean_diag_dist","H1_mean_lifetime",
    "H1_median_lifetime","H1_min_lifetime","H1_persistence_entropy",
    "H1_q50","H1_q75","H1_q90","H1_q95","H1_q99",
    "H1_std_birth","H1_std_death","H1_std_lifetime",
    "H1_sum_diag_dist","H1_sum_lifetime",
    "H1_tail_share_q80","H1_tail_share_q90","H1_tail_share_q95",
    "H1_to_H0_betti_ratio","H1_to_H0_entropy_ratio",
    "PETE_p1.6_q0.5"
]

FEATURE_NAMES = ROBUST_FEATURES[:]  # now user can pick any of the 77

# ==========================================================
# 4. MAIN: PICK ONE FEATURE -> VEAD -> NORMALIZE -> WRITE
# ==========================================================
def run():
    print("\nAvailable TDA features (77):")
    for idx, name in enumerate(FEATURE_NAMES):
        print(f"  {idx:2d} -> {name}")
    choice = input("Select feature by index or name (default: H0_bottleneck): ").strip()

    selected_feature = "H0_bottleneck"
    if choice == "":
        pass
    elif choice.isdigit():
        idx = int(choice)
        if 0 <= idx < len(FEATURE_NAMES):
            selected_feature = FEATURE_NAMES[idx]
        else:
            print(f"Index {idx} out of range, using default H0_bottleneck.")
    else:
        if choice in FEATURE_NAMES:
            selected_feature = choice
        else:
            print(f"Feature '{choice}' not recognized, using default H0_bottleneck.")

    print(f"\n>>> Using TDA feature: {selected_feature}")
    print(f">>> VEAD mode   : {MODE}")
    print(f">>> KV, KA      : {KV}, {KA}\n")

    files = glob.glob(os.path.join(INPUT_DIR, "**", "*.csv"), recursive=True)
    print(f"   Found {len(files)} data files in '{INPUT_DIR}'")

    for filepath in files:
        if ".ipynb_checkpoints" in filepath:
            continue

        try:
            df = pd.read_csv(filepath)
            df.columns = [c.strip().lower() for c in df.columns]
            if "value" not in df.columns or "timestamp" not in df.columns:
                continue

            vals = pd.to_numeric(df["value"], errors="coerce").astype(float).to_numpy()
            n = len(vals)

            rows = []
            for i in range(WINDOW_SIZE - 1, n):
                w = vals[i - WINDOW_SIZE + 1 : i + 1]

                try:
                    emb = takens_embed(w, TAU, DIMENSION)
                    dgms = ripser(emb, maxdim=MAXDIM)["dgms"]
                except Exception:
                    dgms = [np.empty((0, 2)), np.empty((0, 2))]

                D0 = dgms[0] if len(dgms) > 0 else np.empty((0, 2))
                D1 = dgms[1] if (MAXDIM >= 1 and len(dgms) > 1) else np.empty((0, 2))

                feats_H0 = compute_features_for_diag(D0, "H0_")
                feats_H1 = compute_features_for_diag(D1, "H1_")
                cross    = compute_cross_dim_features(feats_H0, feats_H1)

                merged = {}
                merged.update(feats_H0)
                merged.update(feats_H1)
                merged.update(cross)
                merged["index"] = i
                rows.append(merged)

            feat_df = pd.DataFrame(rows)
            full = pd.DataFrame(index=np.arange(n))
            if not feat_df.empty:
                feat_df = feat_df.set_index("index")
                full = full.join(feat_df, how="left")

            full = full.replace([np.inf, -np.inf], np.nan).fillna(0.0)

            # ---- SELECT FEATURE SERIES (length n) ----
            series = pd.to_numeric(full.get(selected_feature, 0.0), errors="coerce").astype(float).to_numpy()
            series = np.nan_to_num(series, nan=0.0, posinf=0.0, neginf=0.0)

            # ---- VEAD on selected feature ----
            vead_scores = _vead_series(series, kv=KV, ka=KA, mode=MODE)
            vead_series = pd.Series(vead_scores, index=df.index)

            # ---- Normalize to [0,1] for NAB ----
            mx = float(np.max(vead_series.values)) if len(vead_series) else 0.0
            if not np.isfinite(mx) or mx <= 0:
                final_scores = np.zeros_like(vead_series.values, dtype=float)
            else:
                final_scores = np.clip(vead_series.values / mx, 0.0, 1.0)

            # ---- Write output ----
            rel = os.path.relpath(filepath, INPUT_DIR)
            category = os.path.dirname(rel)
            base_name = os.path.basename(rel)

            out_dir = os.path.join(OUTPUT_DIR, category)
            os.makedirs(out_dir, exist_ok=True)
            out_name = f"{DETECTOR_NAME}_" + base_name
            out_path = os.path.join(out_dir, out_name)

            out_df = pd.DataFrame({
                "timestamp": df["timestamp"],
                "anomaly_score": final_scores
            })
            out_df.to_csv(out_path, index=False)
            print(f"   -> Wrote: {out_path}")

        except Exception as e:
            print(f"   !! Error processing {filepath}: {e}")
            continue

if __name__ == "__main__":
    run()
"""

with open("my_algo.py", "w") as f:
    f.write(tda_code)

print("✅ my_algo.py written.")

# ============================================================
# 4. RUN YOUR DETECTOR ON ALL NAB DATA
# ============================================================
print("--- 4. RUNNING TDA_VEAD_Method ON ALL DATASETS ---")
!python my_algo.py

# ============================================================
# 5. RUN NAB OPTIMIZE + SCORE FOR THIS DETECTOR
# ============================================================
print("--- 5. RUNNING NAB OPTIMIZE + SCORE ---")
!python run.py --optimize --score --detectors TDA_VEAD_Method --normalize


--- 1. CLEAN START ---
Cloning into 'NAB'...
remote: Enumerating objects: 7119, done.
remote: Counting objects: 100% (731/731), done.
remote: Compressing objects: 100% (229/229), done.
remote: Total 7119 (delta 564), reused 502 (delta 502), pack-reused 6388 (from 1)
Receiving objects: 100% (7119/7119), 86.16 MiB | 29.61 MiB/s, done.
Resolving deltas: 100% (4983/4983), done.
Updating files: 100% (1186/1186), done.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 842.1/842.1 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 3.3 MB/s eta 0:00:00
--- 3. WRITING TDA_VEAD_METHOD DETECTOR (VEAD + 77 FEATURES) ---
✅ my_algo.py written.
--- 4. RUNNING TDA_VEAD_Method ON ALL DATASETS ---

Available TDA features (77):
   0 -> H0_Carlsson_f1
   1 -> H0_Carlsson_f3
   2 -> H0_Carlsson_f5
   3 -> H0_L1_lifetime
   4 -> H0_L1_norm
   5 -> H0_L2_lifetime
   6 -> H0_L2_norm
   7 -> H0_Linf_lifetime
   8 -> H0_Linf_norm
   9 -> H0_bottl

# Version 4

In [ ]:
# @title
import os
import shutil

print("--- 1. CLEAN START ---")
os.chdir("/content")

if os.path.exists("NAB"):
    shutil.rmtree("NAB")

!git clone https://github.com/numenta/NAB.git
!pip install -q ripser

os.chdir("/content/NAB")

os.makedirs("config", exist_ok=True)
thr_path = os.path.join("config", "thresholds.json")
if not os.path.exists(thr_path):
    with open(thr_path, "w") as f:
        f.write("{}")

print("--- 3. WRITING TDA_VEAD_METHOD DETECTOR (77 FEATURES) ---")

tda_code = r"""
import os, glob
import numpy as np
import pandas as pd
from ripser import ripser
import warnings
warnings.filterwarnings("ignore")

DETECTOR_NAME = "TDA_VEAD_Method"
INPUT_DIR = "data"
OUTPUT_DIR = os.path.join("results", DETECTOR_NAME)

# ----------------------------------------------------------
# Embedding parameters (fixed in NAB for fairness)
# ----------------------------------------------------------
WINDOW_SIZE = 10
TAU         = 1
DIMENSION   = 3
_EPS        = 1e-12
MAXDIM      = 1   # H0 + H1

# ==========================================================
# 0. VEAD CONFIGURATION (SAME AS YOUR ORIGINAL)
# ==========================================================
KV   = 3.5
KA   = 3.5
MODE = "abs_plateau"  # "strict" | "plateau" | "abs_plateau"

def _vead_series(raw_vals, kv=KV, ka=KA, mode=MODE):
    s = pd.to_numeric(pd.Series(raw_vals, dtype=float), errors="coerce") \
            .interpolate(limit_direction="both")
    v = s.diff(1)
    a = v.diff(1)

    def _zmad(x):
        x = np.asarray(x, dtype=float)
        med = np.nanmedian(x)
        mad = np.nanmedian(np.abs(x - med)) + _EPS
        return (x - med) / mad

    zv = _zmad(v.values)
    za = _zmad(a.values)

    mode = (mode or "strict").lower()
    if mode == "strict":
        zv = np.maximum(0.0, zv)
        za = np.maximum(0.0, za)
    elif mode == "plateau":
        zv = np.where(zv > -0.25, zv, 0.0)
        za = np.where(za > -0.25, za, 0.0)
    elif mode == "abs_plateau":
        zv = np.abs(zv)
        za = np.abs(za)

    score = (kv * zv) * (ka * za)
    return np.nan_to_num(score, nan=0.0, posinf=0.0, neginf=0.0)

# ==========================================================
# 1) TAKENS EMBEDDING
# ==========================================================
def takens_embed(window, time_delay, dimension):
    m = len(window) - (dimension - 1) * time_delay
    if m <= 0:
        raise ValueError("Takens parameters too large for this window.")
    return np.stack(
        [window[j:j + m * time_delay:time_delay] for j in range(dimension)],
        axis=1
    )

# ==========================================================
# 2) DIAGRAM UTILITIES + FEATURE COMPUTATION (H0 + H1)
# ==========================================================
def _clean_diag(diag):
    if diag is None:
        return np.empty((0, 2), dtype=float)
    arr = np.asarray(diag, dtype=float)
    if arr.ndim != 2 or arr.shape[1] != 2 or arr.size == 0:
        return np.empty((0, 2), dtype=float)
    b, d = arr[:, 0], arr[:, 1]
    mask = np.isfinite(b) & np.isfinite(d) & (d > b)
    if not np.any(mask):
        return np.empty((0, 2), dtype=float)
    return np.stack([b[mask], d[mask]], axis=1)

def _lifetimes(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return np.empty(0, dtype=float)
    return np.maximum(arr[:, 1] - arr[:, 0], 0.0)

def _safe_div(a, b):
    return float(a) / float(b + _EPS)

try:
    _trapz = np.trapezoid
except AttributeError:
    _trapz = np.trapz

def _auc_tri_max(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return 0.0
    b_all, d_all = arr[:, 0], arr[:, 1]
    if b_all.min() == d_all.max():
        return 0.0

    grid = np.linspace(b_all.min(), d_all.max(), 64)
    lam1 = np.zeros_like(grid)

    for b, d in arr:
        m = 0.5 * (b + d)
        h = 0.5 * (d - b)
        if h <= 0:
            continue

        left = (grid >= b) & (grid <= m)
        right = (grid >= m) & (grid <= d)

        lam1[left] = np.maximum(lam1[left], (grid[left] - b) * (h / max(m - b, _EPS)))
        lam1[right] = np.maximum(lam1[right], (d - grid[right]) * (h / max(d - m, _EPS)))

    return float(_trapz(lam1, grid))

def _persistence_entropy(diag):
    L = _lifetimes(diag)
    if L.size == 0:
        return 0.0
    S = L.sum()
    if S <= 0:
        return 0.0
    p = L / (S + _EPS)
    return float(-np.sum(p * np.log(p + _EPS)))

def _gini_from_lifetimes(L):
    L = np.sort(L)
    n = len(L)
    if n == 0:
        return 0.0
    S = L.sum()
    if S <= 0:
        return 0.0
    cumL = np.cumsum(L)
    return float(1 + 1/n - 2*np.sum(cumL/(n*S)))

def _tail_share_q(diag, q):
    L = _lifetimes(diag)
    if L.size == 0:
        return 0.0
    qv = np.quantile(L, q)
    return _safe_div(L[L >= qv].sum(), L.sum())

def _birth_death_stats(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return {"mean_birth": 0.0, "mean_death": 0.0, "std_birth": 0.0, "std_death": 0.0}
    b, d = arr[:, 0], arr[:, 1]
    return {
        "mean_birth": float(b.mean()),
        "mean_death": float(d.mean()),
        "std_birth": float(b.std(ddof=0)),
        "std_death": float(d.std(ddof=0)),
    }

def _diag_distance_stats(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return {"mean_diag_dist": 0.0, "max_diag_dist": 0.0, "sum_diag_dist": 0.0}
    b, d = arr[:, 0], arr[:, 1]
    dist = (d - b) / np.sqrt(2.0)
    return {
        "mean_diag_dist": float(dist.mean()),
        "max_diag_dist": float(dist.max()),
        "sum_diag_dist": float(dist.sum()),
    }

def _centroid_xy(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return {"centroid_x": 0.0, "centroid_y": 0.0}
    b, d = arr[:, 0], arr[:, 1]
    L = np.maximum(d - b, 0.0)
    S = L.sum()
    if S <= 0:
        return {"centroid_x": 0.0, "centroid_y": 0.0}
    return {
        "centroid_x": float(np.sum(b * L) / (S + _EPS)),
        "centroid_y": float(np.sum(d * L) / (S + _EPS)),
    }

def _lifetimes_stats(diag):
    L = _lifetimes(diag)
    if L.size == 0:
        return {"count": 0, "sum": 0.0, "mean": 0.0, "median": 0.0, "std": 0.0,
                "min": 0.0, "max": 0.0, "L1": 0.0, "L2": 0.0, "Linf": 0.0}
    return {
        "count": int(L.size),
        "sum": float(L.sum()),
        "mean": float(L.mean()),
        "median": float(np.median(L)),
        "std": float(L.std(ddof=0)),
        "min": float(L.min()),
        "max": float(L.max()),
        "L1": float(np.sum(np.abs(L))),
        "L2": float(np.sqrt(np.sum(L**2))),
        "Linf": float(np.max(np.abs(L))),
    }

def _lifetimes_quantiles(diag):
    L = _lifetimes(diag)
    if L.size == 0:
        return {"q50": 0.0, "q75": 0.0, "q90": 0.0, "q95": 0.0, "q99": 0.0}
    return {
        "q50": float(np.quantile(L, 0.50)),
        "q75": float(np.quantile(L, 0.75)),
        "q90": float(np.quantile(L, 0.90)),
        "q95": float(np.quantile(L, 0.95)),
        "q99": float(np.quantile(L, 0.99)),
    }

def _carlsson_coordinates(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return {f"f{k}": 0.0 for k in range(1, 6)}
    b, d = arr[:, 0], arr[:, 1]
    L = np.maximum(d - b, 0.0)
    S = L.sum()
    if S <= 0:
        return {f"f{k}": 0.0 for k in range(1, 6)}
    return {
        "f1": float(L.sum()),
        "f2": float(np.sum(b * L)),
        "f3": float(np.sum(d * L)),
        "f4": float(np.sum(b**2 * L)),
        "f5": float(np.sum(d**2 * L)),
    }

def _sum_centroid_radial(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return 0.0
    b, d = arr[:, 0], arr[:, 1]
    L = np.maximum(d - b, 0.0)
    S = L.sum()
    if S <= 0:
        return 0.0
    radial = (b + d) / np.sqrt(2.0)
    return _safe_div(np.sum(np.abs(radial) * L), S)

def _pete(diag, p=1.6, q=0.5):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return 0.0
    b, d = arr[:, 0], arr[:, 1]
    L = np.maximum(d - b, 0.0)
    S = L.sum()
    if S <= 0:
        return 0.0
    radial = (b + d) / np.sqrt(2.0)
    return _safe_div(np.sum((L**p) * (np.abs(radial)**q)), S)

def compute_features_for_diag(diag, prefix):
    feats = {}

    Ls = _lifetimes_stats(diag)
    feats[f"{prefix}count_lifetime"] = float(Ls["count"])
    feats[f"{prefix}sum_lifetime"]   = float(Ls["sum"])
    feats[f"{prefix}mean_lifetime"]  = float(Ls["mean"])
    feats[f"{prefix}median_lifetime"]= float(Ls["median"])
    feats[f"{prefix}std_lifetime"]   = float(Ls["std"])
    feats[f"{prefix}min_lifetime"]   = float(Ls["min"])
    feats[f"{prefix}max_lifetime"]   = float(Ls["max"])

    feats[f"{prefix}L1_lifetime"]    = float(Ls["L1"])
    feats[f"{prefix}L2_lifetime"]    = float(Ls["L2"])
    feats[f"{prefix}Linf_lifetime"]  = float(Ls["Linf"])

    feats[f"{prefix}L1_norm"]        = float(Ls["L1"])
    feats[f"{prefix}L2_norm"]        = float(Ls["L2"])
    feats[f"{prefix}Linf_norm"]      = float(Ls["Linf"])

    feats[f"{prefix}betti"]          = float(Ls["count"])
    feats[f"{prefix}energy_concentration"] = _safe_div(Ls["L2"], Ls["L1"])
    feats[f"{prefix}dominance_share"]      = _safe_div(Ls["Linf"], Ls["L1"])

    feats[f"{prefix}persistence_entropy"]  = _persistence_entropy(diag)

    bd = _birth_death_stats(diag)
    for k, v in bd.items():
        feats[f"{prefix}{k}"] = float(v)

    dd = _diag_distance_stats(diag)
    for k, v in dd.items():
        feats[f"{prefix}{k}"] = float(v)

    cxy = _centroid_xy(diag)
    feats[f"{prefix}centroid_x"] = float(cxy["centroid_x"])
    feats[f"{prefix}centroid_y"] = float(cxy["centroid_y"])

    q = _lifetimes_quantiles(diag)
    for k, v in q.items():
        feats[f"{prefix}{k}"] = float(v)

    tail80 = _tail_share_q(diag, 0.80)
    tail90 = _tail_share_q(diag, 0.90)
    tail95 = _tail_share_q(diag, 0.95)

    feats[f"{prefix}tail_share_q80"] = float(tail80)
    feats[f"{prefix}tail_share_q90"] = float(tail90)
    feats[f"{prefix}tail_share_q95"] = float(tail95)
    feats[f"{prefix}tail_curvature_80_90"] = float(tail90 - tail80)

    L = _lifetimes(diag)
    feats[f"{prefix}gini"] = float(_gini_from_lifetimes(L))

    cc = _carlsson_coordinates(diag)
    feats[f"{prefix}Carlsson_f1"] = float(cc["f1"])
    feats[f"{prefix}Carlsson_f2"] = float(cc["f2"])
    feats[f"{prefix}Carlsson_f3"] = float(cc["f3"])
    feats[f"{prefix}Carlsson_f4"] = float(cc["f4"])
    feats[f"{prefix}Carlsson_f5"] = float(cc["f5"])

    # H0-only extras used in your 77-feature code
    if prefix == "H0_":
        A = _auc_tri_max(diag)
        feats["H0_ratio_auc_L1_to_sum"] = _safe_div(A, Ls["sum"])
        feats["H0_ratio_auc_to_max"]    = _safe_div(A, Ls["max"])
        feats["H0_ratio_auc_to_l2"]     = _safe_div(A, Ls["L2"])
        feats["H0_bottleneck"]          = float(Ls["max"])
        feats["H0_sum_centroid"]        = float(_sum_centroid_radial(diag))
        feats["PETE_p1.6_q0.5"]         = float(_pete(diag, p=1.6, q=0.5))
        feats["H0_energy_concentration"]= _safe_div(Ls["L2"], Ls["sum"])
        feats["H0_dominance_share"]     = _safe_div(Ls["Linf"], Ls["sum"])
        feats["H0_tail_curvature_80_90"]= float(tail90 - tail80)
        feats["H0_centroid_to_energy"]  = _safe_div(feats["H0_sum_centroid"], Ls["L2"])
        feats["H0_gini"]                = float(feats.get("H0_gini", 0.0))

    return feats

def compute_cross_dim_features(feats_H0, feats_H1):
    out = {}
    def g(d, k): return float(d.get(k, 0.0))
    out["H1_to_H0_betti_ratio"]   = _safe_div(g(feats_H1, "H1_betti"), g(feats_H0, "H0_betti"))
    out["H1_to_H0_entropy_ratio"] = _safe_div(g(feats_H1, "H1_persistence_entropy"), g(feats_H0, "H0_persistence_entropy"))
    return out

# ==========================================================
# 3) THE 77 ROBUST FEATURES (EXACT LIST)
# ==========================================================
ROBUST_FEATURES = [
    "H0_Carlsson_f1","H0_Carlsson_f3","H0_Carlsson_f5",
    "H0_L1_lifetime","H0_L1_norm","H0_L2_lifetime","H0_L2_norm",
    "H0_Linf_lifetime","H0_Linf_norm","H0_bottleneck","H0_centroid_to_energy",
    "H0_centroid_y","H0_dominance_share","H0_energy_concentration","H0_gini",
    "H0_max_diag_dist","H0_max_lifetime","H0_mean_death","H0_mean_diag_dist",
    "H0_mean_lifetime","H0_median_lifetime","H0_min_lifetime","H0_persistence_entropy",
    "H0_q50","H0_q75","H0_q90","H0_q95","H0_q99","H0_ratio_auc_L1_to_sum",
    "H0_ratio_auc_to_l2","H0_ratio_auc_to_max","H0_std_death","H0_std_lifetime",
    "H0_sum_centroid","H0_sum_diag_dist","H0_sum_lifetime","H0_tail_curvature_80_90",
    "H0_tail_share_q80","H0_tail_share_q90","H0_tail_share_q95",
    "H1_Carlsson_f1","H1_Carlsson_f2","H1_Carlsson_f3",
    "H1_L1_lifetime","H1_L1_norm","H1_L2_lifetime","H1_L2_norm",
    "H1_Linf_lifetime","H1_Linf_norm","H1_betti","H1_count_lifetime",
    "H1_dominance_share","H1_energy_concentration","H1_gini",
    "H1_max_diag_dist","H1_max_lifetime","H1_mean_diag_dist","H1_mean_lifetime",
    "H1_median_lifetime","H1_min_lifetime","H1_persistence_entropy",
    "H1_q50","H1_q75","H1_q90","H1_q95","H1_q99",
    "H1_std_birth","H1_std_death","H1_std_lifetime",
    "H1_sum_diag_dist","H1_sum_lifetime",
    "H1_tail_share_q80","H1_tail_share_q90","H1_tail_share_q95",
    "H1_to_H0_betti_ratio","H1_to_H0_entropy_ratio",
    "PETE_p1.6_q0.5"
]

def _zmad_vector(x):
    x = np.asarray(x, dtype=float)
    med = np.nanmedian(x)
    mad = np.nanmedian(np.abs(x - med)) + _EPS
    return (x - med) / mad

def combine_features_to_score(feature_row: dict, feature_names):
    # feature_row: {feature_name: value, ...} for ONE time index
    vals = []
    for c in feature_names:
        v = float(feature_row.get(c, 0.0))
        if not np.isfinite(v):
            v = 0.0
        vals.append(v)

    # robust z across features? (not meaningful for a single row)
    # Instead we follow your original approach: zMAD per feature over time.
    # But to keep your exact logic, we will aggregate ABS(zMAD) computed later on a time-series matrix.
    # -> This function is not used; kept for compatibility.
    return float(np.nanmean(np.abs(np.asarray(vals, dtype=float))))

def combine_features_matrix_to_score(feature_matrix: pd.DataFrame, feature_names):
    cols = [c for c in feature_names if c in feature_matrix.columns]
    if len(cols) == 0:
        return np.zeros(len(feature_matrix), dtype=float)

    Z = []
    for c in cols:
        v = pd.to_numeric(feature_matrix[c], errors="coerce").astype(float).to_numpy()
        v = np.nan_to_num(v, nan=0.0, posinf=0.0, neginf=0.0)
        z = _zmad_vector(v)
        Z.append(np.abs(z))

    Z = np.vstack(Z).T
    raw = np.nanmean(Z, axis=1)
    raw = np.nan_to_num(raw, nan=0.0, posinf=0.0, neginf=0.0)

    mn, mx = float(np.min(raw)), float(np.max(raw))
    if mx - mn <= _EPS:
        return np.zeros_like(raw)
    return (raw - mn) / (mx - mn + _EPS)

# ==========================================================
# 4) MAIN RUN
# ==========================================================
def run():
    files = glob.glob(os.path.join(INPUT_DIR, "**", "*.csv"), recursive=True)
    print(f"   Found {len(files)} data files in '{INPUT_DIR}'")

    for filepath in files:
        if ".ipynb_checkpoints" in filepath:
            continue

        try:
            df = pd.read_csv(filepath)
            df.columns = [c.strip().lower() for c in df.columns]
            if "value" not in df.columns or "timestamp" not in df.columns:
                continue

            vals = pd.to_numeric(df["value"], errors="coerce").astype(float).to_numpy()
            n = len(vals)

            # Build per-index feature rows
            rows = []
            for i in range(WINDOW_SIZE - 1, n):
                w = vals[i - WINDOW_SIZE + 1 : i + 1]

                try:
                    emb = takens_embed(w, TAU, DIMENSION)
                    dgms = ripser(emb, maxdim=MAXDIM)["dgms"]
                except Exception:
                    dgms = [np.empty((0, 2)), np.empty((0, 2))]

                D0 = dgms[0] if len(dgms) > 0 else np.empty((0, 2))
                D1 = dgms[1] if (MAXDIM >= 1 and len(dgms) > 1) else np.empty((0, 2))

                feats_H0 = compute_features_for_diag(D0, "H0_")
                feats_H1 = compute_features_for_diag(D1, "H1_")
                cross    = compute_cross_dim_features(feats_H0, feats_H1)

                merged = {}
                merged.update(feats_H0)
                merged.update(feats_H1)
                merged.update(cross)
                merged["index"] = i
                rows.append(merged)

            feat_df = pd.DataFrame(rows)
            full = pd.DataFrame(index=np.arange(n))
            if not feat_df.empty:
                feat_df = feat_df.set_index("index")
                full = full.join(feat_df, how="left")

            full = full.replace([np.inf, -np.inf], np.nan).fillna(0.0)

            # 1) Combine 77 features -> ONE raw score in [0,1]
            tda_scores_01 = combine_features_matrix_to_score(full, ROBUST_FEATURES)

            # 2) Apply VEAD on the combined TDA score
            vead_scores = _vead_series(tda_scores_01, kv=KV, ka=KA, mode=MODE)

            # 3) Normalize VEAD output to [0,1] for NAB
            mx = float(np.max(vead_scores)) if vead_scores.size else 0.0
            if not np.isfinite(mx) or mx <= 0:
                final_scores = np.zeros(n, dtype=float)
            else:
                final_scores = np.clip(vead_scores / mx, 0.0, 1.0)

            # Output path
            rel = os.path.relpath(filepath, INPUT_DIR)
            category = os.path.dirname(rel)
            base_name = os.path.basename(rel)

            out_dir = os.path.join(OUTPUT_DIR, category)
            os.makedirs(out_dir, exist_ok=True)
            out_name = f"{DETECTOR_NAME}_" + base_name
            out_path = os.path.join(out_dir, out_name)

            out_df = pd.DataFrame({
                "timestamp": df["timestamp"],
                "anomaly_score": final_scores
            })
            out_df.to_csv(out_path, index=False)
            print(f"   -> Wrote: {out_path}")

        except Exception as e:
            print(f"   !! Error processing {filepath}: {e}")
            continue

if __name__ == "__main__":
    run()
"""

with open("my_algo.py", "w") as f:
    f.write(tda_code)

print("✅ my_algo.py written.")

print("--- 4. RUNNING TDA_VEAD_Method ON ALL DATASETS ---")
!python my_algo.py

print("--- 5. RUNNING NAB OPTIMIZE + SCORE ---")
!python run.py --optimize --score --detectors TDA_VEAD_Method --normalize


--- 1. CLEAN START ---
Cloning into 'NAB'...
remote: Enumerating objects: 7119, done.
remote: Counting objects: 100% (699/699), done.
remote: Compressing objects: 100% (204/204), done.
remote: Total 7119 (delta 552), reused 495 (delta 495), pack-reused 6420 (from 1)
Receiving objects: 100% (7119/7119), 86.13 MiB | 21.23 MiB/s, done.
Resolving deltas: 100% (5001/5001), done.
Updating files: 100% (1186/1186), done.
--- 3. WRITING TDA_VEAD_METHOD DETECTOR (77 FEATURES) ---
✅ my_algo.py written.
--- 4. RUNNING TDA_VEAD_Method ON ALL DATASETS ---
   Found 58 data files in 'data'
   -> Wrote: results/TDA_VEAD_Method/realAdExchange/TDA_VEAD_Method_exchange-4_cpc_results.csv
   -> Wrote: results/TDA_VEAD_Method/realAdExchange/TDA_VEAD_Method_exchange-2_cpc_results.csv
   -> Wrote: results/TDA_VEAD_Method/realAdExchange/TDA_VEAD_Method_exchange-2_cpm_results.csv
   -> Wrote: results/TDA_VEAD_Method/realAdExchange/TDA_VEAD_Method_exchange-3_cpm_results.csv
   -> Wrote: results/TDA_VEAD_Method/re

# Version 5 _ full flow w 5 top selected

In [ ]:
# @title
import os
import shutil
import glob
import numpy as np
import pandas as pd

# ============================================================
# 1. CLEAN START & CLONE NAB
# ============================================================
print("--- 1. CLEAN START ---")
os.chdir("/content")

if os.path.exists("NAB"):
    shutil.rmtree("NAB")

!git clone https://github.com/numenta/NAB.git
!pip install -q ripser

os.chdir("/content/NAB")

os.makedirs("config", exist_ok=True)
thr_path = os.path.join("config", "thresholds.json")
if not os.path.exists(thr_path):
    with open(thr_path, "w") as f:
        f.write("{}")

# ============================================================
# 3. WRITE TDA_VEAD_METHOD (my_algo.py)  [VEAD + 77 FEATURES]
# ============================================================
print("--- 3. WRITING TDA_VEAD_METHOD DETECTOR (VEAD + 77 FEATURES) ---")

tda_code = r"""
import os
import glob
import numpy as np
import pandas as pd
from ripser import ripser
import warnings

warnings.filterwarnings("ignore")

DETECTOR_NAME = "TDA_VEAD_Method"
INPUT_DIR = "data"
OUTPUT_DIR = os.path.join("results", DETECTOR_NAME)

# ----------------------------------------------------------
# Embedding parameters (fixed in NAB for fairness)
# ----------------------------------------------------------
WINDOW_SIZE = 14
TAU         = 1
DIMENSION   = 7
_EPS        = 1e-12

MAXDIM = 1  # H0 + H1

# ==========================================================
# 0. VEAD CONFIGURATION (UNCHANGED)
# ==========================================================
KV   = 3.5
KA   = 3.5
MODE = "abs_plateau"  # "strict" | "plateau" | "abs_plateau"

def _vead_series(raw_vals, kv=KV, ka=KA, mode=MODE):
    # 1) Convert to pandas Series & interpolate
    s = pd.to_numeric(pd.Series(raw_vals, dtype=float), errors="coerce") \
            .interpolate(limit_direction="both")

    # 2) Velocity and acceleration
    v = s.diff(1)
    a = v.diff(1)

    # 3) Robust Z-score with MAD
    def _zmad(x):
        x = np.asarray(x, dtype=float)
        med = np.nanmedian(x)
        mad = np.nanmedian(np.abs(x - med)) + 1e-12
        return (x - med) / mad

    zv = _zmad(v.values)
    za = _zmad(a.values)

    # 4) Mode logic
    mode = (mode or "strict").lower()
    if mode == "strict":
        zv = np.maximum(0.0, zv)
        za = np.maximum(0.0, za)
    elif mode == "plateau":
        zv = np.where(zv > -0.25, zv, 0.0)
        za = np.where(za > -0.25, za, 0.0)
    elif mode == "abs_plateau":
        zv = np.abs(zv)
        za = np.abs(za)

    score = (kv * zv) * (ka * za)
    return np.nan_to_num(score, nan=0.0, posinf=0.0, neginf=0.0)

# ==========================================================
# 1. TAKENS EMBEDDING
# ==========================================================
def takens_embed(window, time_delay, dimension):
    m = len(window) - (dimension - 1) * time_delay
    if m <= 0:
        raise ValueError("Takens parameters too large for this window.")
    return np.stack(
        [window[j:j + m * time_delay:time_delay] for j in range(dimension)],
        axis=1
    )

# ==========================================================
# 2. PERSISTENCE DIAGRAM UTILITIES + FEATURE FUNCTIONS
# ==========================================================
def _clean_diag(diag):
    if diag is None:
        return np.empty((0, 2), dtype=float)
    arr = np.asarray(diag, dtype=float)
    if arr.ndim != 2 or arr.shape[1] != 2 or arr.size == 0:
        return np.empty((0, 2), dtype=float)
    b, d = arr[:, 0], arr[:, 1]
    mask = np.isfinite(b) & np.isfinite(d) & (d > b)
    if not np.any(mask):
        return np.empty((0, 2), dtype=float)
    return np.stack([b[mask], d[mask]], axis=1)

def _lifetimes(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return np.empty(0, dtype=float)
    return np.maximum(arr[:, 1] - arr[:, 0], 0.0)

def _safe_div(a, b):
    return float(a) / float(b + _EPS)

try:
    _trapz = np.trapezoid
except AttributeError:
    _trapz = np.trapz

def _auc_tri_max(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return 0.0
    b_all, d_all = arr[:, 0], arr[:, 1]
    if b_all.min() == d_all.max():
        return 0.0

    grid = np.linspace(b_all.min(), d_all.max(), 64)
    lam1 = np.zeros_like(grid)

    for b, d in arr:
        m = 0.5 * (b + d)
        h = 0.5 * (d - b)
        if h <= 0:
            continue

        left = (grid >= b) & (grid <= m)
        right = (grid >= m) & (grid <= d)

        lam1[left] = np.maximum(lam1[left], (grid[left] - b) * (h / max(m - b, _EPS)))
        lam1[right] = np.maximum(lam1[right], (d - grid[right]) * (h / max(d - m, _EPS)))

    return float(_trapz(lam1, grid))

def _persistence_entropy(diag):
    L = _lifetimes(diag)
    if L.size == 0:
        return 0.0
    S = L.sum()
    if S <= 0:
        return 0.0
    p = L / (S + _EPS)
    return float(-np.sum(p * np.log(p + _EPS)))

def _gini_from_lifetimes(L):
    L = np.sort(L)
    n = len(L)
    if n == 0:
        return 0.0
    S = L.sum()
    if S <= 0:
        return 0.0
    cumL = np.cumsum(L)
    return float(1 + 1/n - 2*np.sum(cumL/(n*S)))

def _tail_share_q(diag, q):
    L = _lifetimes(diag)
    if L.size == 0:
        return 0.0
    qv = np.quantile(L, q)
    return _safe_div(L[L >= qv].sum(), L.sum())

def _birth_death_stats(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return {"mean_birth": 0.0, "mean_death": 0.0, "std_birth": 0.0, "std_death": 0.0}
    b, d = arr[:, 0], arr[:, 1]
    return {
        "mean_birth": float(b.mean()),
        "mean_death": float(d.mean()),
        "std_birth": float(b.std(ddof=0)),
        "std_death": float(d.std(ddof=0)),
    }

def _diag_distance_stats(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return {"mean_diag_dist": 0.0, "max_diag_dist": 0.0, "sum_diag_dist": 0.0}
    b, d = arr[:, 0], arr[:, 1]
    dist = (d - b) / np.sqrt(2.0)
    return {
        "mean_diag_dist": float(dist.mean()),
        "max_diag_dist": float(dist.max()),
        "sum_diag_dist": float(dist.sum()),
    }

def _centroid_xy(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return {"centroid_x": 0.0, "centroid_y": 0.0}
    b, d = arr[:, 0], arr[:, 1]
    L = np.maximum(d - b, 0.0)
    S = L.sum()
    if S <= 0:
        return {"centroid_x": 0.0, "centroid_y": 0.0}
    return {
        "centroid_x": float(np.sum(b * L) / (S + _EPS)),
        "centroid_y": float(np.sum(d * L) / (S + _EPS)),
    }

def _lifetimes_stats(diag):
    L = _lifetimes(diag)
    if L.size == 0:
        return {
            "count": 0, "sum": 0.0, "mean": 0.0, "median": 0.0, "std": 0.0,
            "min": 0.0, "max": 0.0, "L1": 0.0, "L2": 0.0, "Linf": 0.0
        }
    return {
        "count": int(L.size),
        "sum": float(L.sum()),
        "mean": float(L.mean()),
        "median": float(np.median(L)),
        "std": float(L.std(ddof=0)),
        "min": float(L.min()),
        "max": float(L.max()),
        "L1": float(np.sum(np.abs(L))),
        "L2": float(np.sqrt(np.sum(L**2))),
        "Linf": float(np.max(np.abs(L))),
    }

def _lifetimes_quantiles(diag):
    L = _lifetimes(diag)
    if L.size == 0:
        return {"q50": 0.0, "q75": 0.0, "q90": 0.0, "q95": 0.0, "q99": 0.0}
    return {
        "q50": float(np.quantile(L, 0.50)),
        "q75": float(np.quantile(L, 0.75)),
        "q90": float(np.quantile(L, 0.90)),
        "q95": float(np.quantile(L, 0.95)),
        "q99": float(np.quantile(L, 0.99)),
    }

def _carlsson_coordinates(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return {f"f{k}": 0.0 for k in range(1, 6)}
    b, d = arr[:, 0], arr[:, 1]
    L = np.maximum(d - b, 0.0)
    S = L.sum()
    if S <= 0:
        return {f"f{k}": 0.0 for k in range(1, 6)}
    return {
        "f1": float(L.sum()),
        "f2": float(np.sum(b * L)),
        "f3": float(np.sum(d * L)),
        "f4": float(np.sum(b**2 * L)),
        "f5": float(np.sum(d**2 * L)),
    }

def _sum_centroid_radial(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return 0.0
    b, d = arr[:, 0], arr[:, 1]
    L = np.maximum(d - b, 0.0)
    S = L.sum()
    if S <= 0:
        return 0.0
    radial = (b + d) / np.sqrt(2.0)
    return _safe_div(np.sum(np.abs(radial) * L), S)

def _pete(diag, p=1.6, q=0.5):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return 0.0
    b, d = arr[:, 0], arr[:, 1]
    L = np.maximum(d - b, 0.0)
    S = L.sum()
    if S <= 0:
        return 0.0
    radial = (b + d) / np.sqrt(2.0)
    return _safe_div(np.sum((L**p) * (np.abs(radial)**q)), S)

def compute_features_for_diag(diag, prefix):
    feats = {}

    Ls = _lifetimes_stats(diag)
    feats[f"{prefix}count_lifetime"] = float(Ls["count"])
    feats[f"{prefix}sum_lifetime"]   = float(Ls["sum"])
    feats[f"{prefix}mean_lifetime"]  = float(Ls["mean"])
    feats[f"{prefix}median_lifetime"]= float(Ls["median"])
    feats[f"{prefix}std_lifetime"]   = float(Ls["std"])
    feats[f"{prefix}min_lifetime"]   = float(Ls["min"])
    feats[f"{prefix}max_lifetime"]   = float(Ls["max"])

    feats[f"{prefix}L1_lifetime"]    = float(Ls["L1"])
    feats[f"{prefix}L2_lifetime"]    = float(Ls["L2"])
    feats[f"{prefix}Linf_lifetime"]  = float(Ls["Linf"])

    feats[f"{prefix}L1_norm"]        = float(Ls["L1"])
    feats[f"{prefix}L2_norm"]        = float(Ls["L2"])
    feats[f"{prefix}Linf_norm"]      = float(Ls["Linf"])

    feats[f"{prefix}betti"]          = float(Ls["count"])
    feats[f"{prefix}energy_concentration"] = _safe_div(Ls["L2"], Ls["L1"])
    feats[f"{prefix}dominance_share"]      = _safe_div(Ls["Linf"], Ls["L1"])

    feats[f"{prefix}persistence_entropy"]  = _persistence_entropy(diag)

    bd = _birth_death_stats(diag)
    for k, v in bd.items():
        feats[f"{prefix}{k}"] = float(v)

    dd = _diag_distance_stats(diag)
    for k, v in dd.items():
        feats[f"{prefix}{k}"] = float(v)

    cxy = _centroid_xy(diag)
    feats[f"{prefix}centroid_x"] = float(cxy["centroid_x"])
    feats[f"{prefix}centroid_y"] = float(cxy["centroid_y"])

    q = _lifetimes_quantiles(diag)
    for k, v in q.items():
        feats[f"{prefix}{k}"] = float(v)

    tail80 = _tail_share_q(diag, 0.80)
    tail90 = _tail_share_q(diag, 0.90)
    tail95 = _tail_share_q(diag, 0.95)

    feats[f"{prefix}tail_share_q80"] = float(tail80)
    feats[f"{prefix}tail_share_q90"] = float(tail90)
    feats[f"{prefix}tail_share_q95"] = float(tail95)
    feats[f"{prefix}tail_curvature_80_90"] = float(tail90 - tail80)

    L = _lifetimes(diag)
    feats[f"{prefix}gini"] = float(_gini_from_lifetimes(L))

    cc = _carlsson_coordinates(diag)
    feats[f"{prefix}Carlsson_f1"] = float(cc["f1"])
    feats[f"{prefix}Carlsson_f2"] = float(cc["f2"])
    feats[f"{prefix}Carlsson_f3"] = float(cc["f3"])
    feats[f"{prefix}Carlsson_f4"] = float(cc["f4"])
    feats[f"{prefix}Carlsson_f5"] = float(cc["f5"])

    if prefix == "H0_":
        A = _auc_tri_max(diag)
        feats["H0_ratio_auc_L1_to_sum"] = _safe_div(A, Ls["sum"])
        feats["H0_ratio_auc_to_max"]    = _safe_div(A, Ls["max"])
        feats["H0_ratio_auc_to_l2"]     = _safe_div(A, Ls["L2"])
        feats["H0_bottleneck"]          = float(Ls["max"])
        feats["H0_sum_centroid"]        = float(_sum_centroid_radial(diag))
        feats["PETE_p1.6_q0.5"]         = float(_pete(diag, p=1.6, q=0.5))
        feats["H0_energy_concentration"]= _safe_div(Ls["L2"], Ls["sum"])
        feats["H0_dominance_share"]     = _safe_div(Ls["Linf"], Ls["sum"])
        feats["H0_tail_curvature_80_90"]= float(tail90 - tail80)
        feats["H0_centroid_to_energy"]  = _safe_div(feats["H0_sum_centroid"], Ls["L2"])
        feats["H0_gini"]                = float(feats["H0_gini"])

    return feats

def compute_cross_dim_features(feats_H0, feats_H1):
    out = {}
    def g(d, k): return float(d.get(k, 0.0))
    out["H1_to_H0_betti_ratio"]   = _safe_div(g(feats_H1, "H1_betti"), g(feats_H0, "H0_betti"))
    out["H1_to_H0_entropy_ratio"] = _safe_div(g(feats_H1, "H1_persistence_entropy"), g(feats_H0, "H0_persistence_entropy"))
    return out

# ==========================================================
# 3. ROBUST FEATURES LIST (77)
# ==========================================================
ROBUST_FEATURES = [
    "H0_Carlsson_f1","H0_Carlsson_f3","H0_Carlsson_f5",
    "H0_L1_lifetime","H0_L1_norm","H0_L2_lifetime","H0_L2_norm",
    "H0_Linf_lifetime","H0_Linf_norm","H0_bottleneck","H0_centroid_to_energy",
    "H0_centroid_y","H0_dominance_share","H0_energy_concentration","H0_gini",
    "H0_max_diag_dist","H0_max_lifetime","H0_mean_death","H0_mean_diag_dist",
    "H0_mean_lifetime","H0_median_lifetime","H0_min_lifetime","H0_persistence_entropy",
    "H0_q50","H0_q75","H0_q90","H0_q95","H0_q99","H0_ratio_auc_L1_to_sum",
    "H0_ratio_auc_to_l2","H0_ratio_auc_to_max","H0_std_death","H0_std_lifetime",
    "H0_sum_centroid","H0_sum_diag_dist","H0_sum_lifetime","H0_tail_curvature_80_90",
    "H0_tail_share_q80","H0_tail_share_q90","H0_tail_share_q95",
    "H1_Carlsson_f1","H1_Carlsson_f2","H1_Carlsson_f3",
    "H1_L1_lifetime","H1_L1_norm","H1_L2_lifetime","H1_L2_norm",
    "H1_Linf_lifetime","H1_Linf_norm","H1_betti","H1_count_lifetime",
    "H1_dominance_share","H1_energy_concentration","H1_gini",
    "H1_max_diag_dist","H1_max_lifetime","H1_mean_diag_dist","H1_mean_lifetime",
    "H1_median_lifetime","H1_min_lifetime","H1_persistence_entropy",
    "H1_q50","H1_q75","H1_q90","H1_q95","H1_q99",
    "H1_std_birth","H1_std_death","H1_std_lifetime",
    "H1_sum_diag_dist","H1_sum_lifetime",
    "H1_tail_share_q80","H1_tail_share_q90","H1_tail_share_q95",
    "H1_to_H0_betti_ratio","H1_to_H0_entropy_ratio",
    "PETE_p1.6_q0.5"
]

FEATURE_NAMES = ROBUST_FEATURES[:]

# ==========================================================
# 4. MAIN
# ==========================================================
def run():
    print("\nAvailable TDA features (77):")
    for idx, name in enumerate(FEATURE_NAMES):
        print(f"  {idx:2d} -> {name}")
    choice = input("Select feature by index or name (default: H0_bottleneck): ").strip()

    selected_feature = "H0_bottleneck"
    if choice == "":
        pass
    elif choice.isdigit():
        idx = int(choice)
        if 0 <= idx < len(FEATURE_NAMES):
            selected_feature = FEATURE_NAMES[idx]
        else:
            print(f"Index {idx} out of range, using default H0_bottleneck.")
    else:
        if choice in FEATURE_NAMES:
            selected_feature = choice
        else:
            print(f"Feature '{choice}' not recognized, using default H0_bottleneck.")

    print(f"\n>>> Using TDA feature: {selected_feature}")
    print(f">>> VEAD mode   : {MODE}")
    print(f">>> KV, KA      : {KV}, {KA}\n")

    files = glob.glob(os.path.join(INPUT_DIR, "**", "*.csv"), recursive=True)
    print(f"   Found {len(files)} data files in '{INPUT_DIR}'")

    for filepath in files:
        if ".ipynb_checkpoints" in filepath:
            continue

        try:
            df = pd.read_csv(filepath)
            df.columns = [c.strip().lower() for c in df.columns]
            if "value" not in df.columns or "timestamp" not in df.columns:
                continue

            vals = pd.to_numeric(df["value"], errors="coerce").astype(float).to_numpy()
            n = len(vals)

            rows = []
            for i in range(WINDOW_SIZE - 1, n):
                w = vals[i - WINDOW_SIZE + 1 : i + 1]

                try:
                    emb = takens_embed(w, TAU, DIMENSION)
                    dgms = ripser(emb, maxdim=MAXDIM)["dgms"]
                except Exception:
                    dgms = [np.empty((0, 2)), np.empty((0, 2))]

                D0 = dgms[0] if len(dgms) > 0 else np.empty((0, 2))
                D1 = dgms[1] if (MAXDIM >= 1 and len(dgms) > 1) else np.empty((0, 2))

                feats_H0 = compute_features_for_diag(D0, "H0_")
                feats_H1 = compute_features_for_diag(D1, "H1_")
                cross    = compute_cross_dim_features(feats_H0, feats_H1)

                merged = {}
                merged.update(feats_H0)
                merged.update(feats_H1)
                merged.update(cross)
                merged["index"] = i
                rows.append(merged)

            feat_df = pd.DataFrame(rows)
            full = pd.DataFrame(index=np.arange(n))
            if not feat_df.empty:
                feat_df = feat_df.set_index("index")
                full = full.join(feat_df, how="left")

            full = full.replace([np.inf, -np.inf], np.nan).fillna(0.0)

            series = pd.to_numeric(full.get(selected_feature, 0.0), errors="coerce").astype(float).to_numpy()
            series = np.nan_to_num(series, nan=0.0, posinf=0.0, neginf=0.0)

            vead_scores = _vead_series(series, kv=KV, ka=KA, mode=MODE)
            vead_series = pd.Series(vead_scores, index=df.index)

            # Normalize [0,1]
            mx = float(np.max(vead_series.values)) if len(vead_series) else 0.0
            if not np.isfinite(mx) or mx <= 0:
                final_scores = np.zeros_like(vead_series.values, dtype=float)
            else:
                final_scores = np.clip(vead_series.values / mx, 0.0, 1.0)

            # Keep only top-5 scores, silence others
            k = 2
            n_scores = len(final_scores)
            if n_scores > 0 and np.max(final_scores) > 0:
                k_eff = min(k, n_scores)
                topk_idx = np.argpartition(final_scores, -k_eff)[-k_eff:]
                sparse_scores = np.zeros_like(final_scores, dtype=float)
                sparse_scores[topk_idx] = final_scores[topk_idx]
                final_scores = sparse_scores

            rel = os.path.relpath(filepath, INPUT_DIR)
            category = os.path.dirname(rel)
            base_name = os.path.basename(rel)

            out_dir = os.path.join(OUTPUT_DIR, category)
            os.makedirs(out_dir, exist_ok=True)
            out_name = f"{DETECTOR_NAME}_" + base_name
            out_path = os.path.join(out_dir, out_name)

            out_df = pd.DataFrame({
                "timestamp": df["timestamp"],
                "anomaly_score": final_scores
            })
            out_df.to_csv(out_path, index=False)
            print(f"   -> Wrote: {out_path}")

        except Exception as e:
            print(f"   !! Error processing {filepath}: {e}")
            continue

if __name__ == "__main__":
    run()
"""

with open("my_algo.py", "w") as f:
    f.write(tda_code)

print("✅ my_algo.py written.")

# ============================================================
# 4. RUN YOUR DETECTOR ON ALL NAB DATA
# ============================================================
print("--- 4. RUNNING TDA_VEAD_Method ON ALL DATASETS ---")
!python my_algo.py

# ============================================================
# 5. RUN NAB OPTIMIZE + SCORE FOR THIS DETECTOR
# ============================================================
print("--- 5. RUNNING NAB OPTIMIZE + SCORE ---")
!python run.py --optimize --score --detectors TDA_VEAD_Method --normalize


--- 1. CLEAN START ---
Cloning into 'NAB'...
remote: Enumerating objects: 7119, done.
remote: Counting objects: 100% (699/699), done.
remote: Compressing objects: 100% (204/204), done.
remote: Total 7119 (delta 552), reused 495 (delta 495), pack-reused 6420 (from 1)
Receiving objects: 100% (7119/7119), 86.13 MiB | 29.12 MiB/s, done.
Resolving deltas: 100% (5001/5001), done.
Updating files: 100% (1186/1186), done.
--- 3. WRITING TDA_VEAD_METHOD DETECTOR (VEAD + 77 FEATURES) ---
✅ my_algo.py written.
--- 4. RUNNING TDA_VEAD_Method ON ALL DATASETS ---

Available TDA features (77):
   0 -> H0_Carlsson_f1
   1 -> H0_Carlsson_f3
   2 -> H0_Carlsson_f5
   3 -> H0_L1_lifetime
   4 -> H0_L1_norm
   5 -> H0_L2_lifetime
   6 -> H0_L2_norm
   7 -> H0_Linf_lifetime
   8 -> H0_Linf_norm
   9 -> H0_bottleneck
  10 -> H0_centroid_to_energy
  11 -> H0_centroid_y
  12 -> H0_dominance_share
  13 -> H0_energy_concentration
  14 -> H0_gini
  15 -> H0_max_diag_dist
  16 -> H0_max_lifetime
  17 -> H0_mean_de

# Version 6 Majority Voting

In [7]:
import os
import shutil
import numpy as np
import pandas as pd

# ============================================================
# 1. CLEAN START & CLONE NAB
# ============================================================
print("--- 1. CLEAN START ---")
os.chdir("/content")

if os.path.exists("NAB"):
    shutil.rmtree("NAB")

!git clone https://github.com/numenta/NAB.git
!pip install -q ripser

os.chdir("/content/NAB")

os.makedirs("config", exist_ok=True)
thr_path = os.path.join("config", "thresholds.json")
if not os.path.exists(thr_path):
    with open(thr_path, "w") as f:
        f.write("{}")

# ============================================================
# 2. WRITE TDA_VEAD_METHOD (my_algo.py)  [VEAD + 77 FEATURES]
#    ENSEMBLE: top-2 per feature -> majority vote -> top-5 = 1 else 0
# ============================================================
print("--- 2. WRITING TDA_VEAD_METHOD (ENSEMBLE VOTING) ---")

tda_code = r"""
import os
import glob
import numpy as np
import pandas as pd
from ripser import ripser
import warnings

warnings.filterwarnings("ignore")

DETECTOR_NAME = "TDA_VEAD_Method"
INPUT_DIR = "data"
OUTPUT_DIR = os.path.join("results", DETECTOR_NAME)

# ----------------------------------------------------------
# Embedding parameters (fixed in NAB for fairness)
# ----------------------------------------------------------
WINDOW_SIZE = 14
TAU         = 1
DIMENSION   = 7
_EPS        = 1e-12
MAXDIM      = 1  # H0 + H1

# ==========================================================
# 0. VEAD CONFIGURATION
# ==========================================================
KV   = 3.5
KA   = 3.5
MODE = "abs_plateau"  # "strict" | "plateau" | "abs_plateau"

def _vead_series(raw_vals, kv=KV, ka=KA, mode=MODE):
    s = pd.to_numeric(pd.Series(raw_vals, dtype=float), errors="coerce") \
            .interpolate(limit_direction="both")

    v = s.diff(1)
    a = v.diff(1)

    def _zmad(x):
        x = np.asarray(x, dtype=float)
        med = np.nanmedian(x)
        mad = np.nanmedian(np.abs(x - med)) + 1e-12
        return (x - med) / mad

    zv = _zmad(v.values)
    za = _zmad(a.values)

    mode = (mode or "strict").lower()
    if mode == "strict":
        zv = np.maximum(0.0, zv)
        za = np.maximum(0.0, za)
    elif mode == "plateau":
        zv = np.where(zv > -0.25, zv, 0.0)
        za = np.where(za > -0.25, za, 0.0)
    elif mode == "abs_plateau":
        zv = np.abs(zv)
        za = np.abs(za)

    score = (kv * zv) * (ka * za)
    return np.nan_to_num(score, nan=0.0, posinf=0.0, neginf=0.0)

# ==========================================================
# 1. TAKENS EMBEDDING
# ==========================================================
def takens_embed(window, time_delay, dimension):
    m = len(window) - (dimension - 1) * time_delay
    if m <= 0:
        raise ValueError("Takens parameters too large for this window.")
    return np.stack(
        [window[j:j + m * time_delay:time_delay] for j in range(dimension)],
        axis=1
    )

# ==========================================================
# 2. PERSISTENCE DIAGRAM UTILITIES + FEATURE FUNCTIONS
# ==========================================================
def _clean_diag(diag):
    if diag is None:
        return np.empty((0, 2), dtype=float)
    arr = np.asarray(diag, dtype=float)
    if arr.ndim != 2 or arr.shape[1] != 2 or arr.size == 0:
        return np.empty((0, 2), dtype=float)
    b, d = arr[:, 0], arr[:, 1]
    mask = np.isfinite(b) & np.isfinite(d) & (d > b)
    if not np.any(mask):
        return np.empty((0, 2), dtype=float)
    return np.stack([b[mask], d[mask]], axis=1)

def _lifetimes(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return np.empty(0, dtype=float)
    return np.maximum(arr[:, 1] - arr[:, 0], 0.0)

def _safe_div(a, b):
    return float(a) / float(b + _EPS)

try:
    _trapz = np.trapezoid
except AttributeError:
    _trapz = np.trapz

def _auc_tri_max(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return 0.0
    b_all, d_all = arr[:, 0], arr[:, 1]
    if b_all.min() == d_all.max():
        return 0.0

    grid = np.linspace(b_all.min(), d_all.max(), 64)
    lam1 = np.zeros_like(grid)

    for b, d in arr:
        m = 0.5 * (b + d)
        h = 0.5 * (d - b)
        if h <= 0:
            continue

        left = (grid >= b) & (grid <= m)
        right = (grid >= m) & (grid <= d)

        lam1[left] = np.maximum(lam1[left], (grid[left] - b) * (h / max(m - b, _EPS)))
        lam1[right] = np.maximum(lam1[right], (d - grid[right]) * (h / max(d - m, _EPS)))

    return float(_trapz(lam1, grid))

def _persistence_entropy(diag):
    L = _lifetimes(diag)
    if L.size == 0:
        return 0.0
    S = L.sum()
    if S <= 0:
        return 0.0
    p = L / (S + _EPS)
    return float(-np.sum(p * np.log(p + _EPS)))

def _gini_from_lifetimes(L):
    L = np.sort(L)
    n = len(L)
    if n == 0:
        return 0.0
    S = L.sum()
    if S <= 0:
        return 0.0
    cumL = np.cumsum(L)
    return float(1 + 1/n - 2*np.sum(cumL/(n*S)))

def _tail_share_q(diag, q):
    L = _lifetimes(diag)
    if L.size == 0:
        return 0.0
    qv = np.quantile(L, q)
    return _safe_div(L[L >= qv].sum(), L.sum())

def _birth_death_stats(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return {"mean_birth": 0.0, "mean_death": 0.0, "std_birth": 0.0, "std_death": 0.0}
    b, d = arr[:, 0], arr[:, 1]
    return {
        "mean_birth": float(b.mean()),
        "mean_death": float(d.mean()),
        "std_birth": float(b.std(ddof=0)),
        "std_death": float(d.std(ddof=0)),
    }

def _diag_distance_stats(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return {"mean_diag_dist": 0.0, "max_diag_dist": 0.0, "sum_diag_dist": 0.0}
    b, d = arr[:, 0], arr[:, 1]
    dist = (d - b) / np.sqrt(2.0)
    return {
        "mean_diag_dist": float(dist.mean()),
        "max_diag_dist": float(dist.max()),
        "sum_diag_dist": float(dist.sum()),
    }

def _centroid_xy(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return {"centroid_x": 0.0, "centroid_y": 0.0}
    b, d = arr[:, 0], arr[:, 1]
    L = np.maximum(d - b, 0.0)
    S = L.sum()
    if S <= 0:
        return {"centroid_x": 0.0, "centroid_y": 0.0}
    return {
        "centroid_x": float(np.sum(b * L) / (S + _EPS)),
        "centroid_y": float(np.sum(d * L) / (S + _EPS)),
    }

def _lifetimes_stats(diag):
    L = _lifetimes(diag)
    if L.size == 0:
        return {
            "count": 0, "sum": 0.0, "mean": 0.0, "median": 0.0, "std": 0.0,
            "min": 0.0, "max": 0.0, "L1": 0.0, "L2": 0.0, "Linf": 0.0
        }
    return {
        "count": int(L.size),
        "sum": float(L.sum()),
        "mean": float(L.mean()),
        "median": float(np.median(L)),
        "std": float(L.std(ddof=0)),
        "min": float(L.min()),
        "max": float(L.max()),
        "L1": float(np.sum(np.abs(L))),
        "L2": float(np.sqrt(np.sum(L**2))),
        "Linf": float(np.max(np.abs(L))),
    }

def _lifetimes_quantiles(diag):
    L = _lifetimes(diag)
    if L.size == 0:
        return {"q50": 0.0, "q75": 0.0, "q90": 0.0, "q95": 0.0, "q99": 0.0}
    return {
        "q50": float(np.quantile(L, 0.50)),
        "q75": float(np.quantile(L, 0.75)),
        "q90": float(np.quantile(L, 0.90)),
        "q95": float(np.quantile(L, 0.95)),
        "q99": float(np.quantile(L, 0.99)),
    }

def _carlsson_coordinates(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return {f"f{k}": 0.0 for k in range(1, 6)}
    b, d = arr[:, 0], arr[:, 1]
    L = np.maximum(d - b, 0.0)
    S = L.sum()
    if S <= 0:
        return {f"f{k}": 0.0 for k in range(1, 6)}
    return {
        "f1": float(L.sum()),
        "f2": float(np.sum(b * L)),
        "f3": float(np.sum(d * L)),
        "f4": float(np.sum(b**2 * L)),
        "f5": float(np.sum(d**2 * L)),
    }

def _sum_centroid_radial(diag):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return 0.0
    b, d = arr[:, 0], arr[:, 1]
    L = np.maximum(d - b, 0.0)
    S = L.sum()
    if S <= 0:
        return 0.0
    radial = (b + d) / np.sqrt(2.0)
    return _safe_div(np.sum(np.abs(radial) * L), S)

def _pete(diag, p=1.6, q=0.5):
    arr = _clean_diag(diag)
    if arr.size == 0:
        return 0.0
    b, d = arr[:, 0], arr[:, 1]
    L = np.maximum(d - b, 0.0)
    S = L.sum()
    if S <= 0:
        return 0.0
    radial = (b + d) / np.sqrt(2.0)
    return _safe_div(np.sum((L**p) * (np.abs(radial)**q)), S)

def compute_features_for_diag(diag, prefix):
    feats = {}

    Ls = _lifetimes_stats(diag)
    feats[f"{prefix}count_lifetime"] = float(Ls["count"])
    feats[f"{prefix}sum_lifetime"]   = float(Ls["sum"])
    feats[f"{prefix}mean_lifetime"]  = float(Ls["mean"])
    feats[f"{prefix}median_lifetime"]= float(Ls["median"])
    feats[f"{prefix}std_lifetime"]   = float(Ls["std"])
    feats[f"{prefix}min_lifetime"]   = float(Ls["min"])
    feats[f"{prefix}max_lifetime"]   = float(Ls["max"])

    feats[f"{prefix}L1_lifetime"]    = float(Ls["L1"])
    feats[f"{prefix}L2_lifetime"]    = float(Ls["L2"])
    feats[f"{prefix}Linf_lifetime"]  = float(Ls["Linf"])

    feats[f"{prefix}L1_norm"]        = float(Ls["L1"])
    feats[f"{prefix}L2_norm"]        = float(Ls["L2"])
    feats[f"{prefix}Linf_norm"]      = float(Ls["Linf"])

    feats[f"{prefix}betti"]          = float(Ls["count"])
    feats[f"{prefix}energy_concentration"] = _safe_div(Ls["L2"], Ls["L1"])
    feats[f"{prefix}dominance_share"]      = _safe_div(Ls["Linf"], Ls["L1"])

    feats[f"{prefix}persistence_entropy"]  = _persistence_entropy(diag)

    bd = _birth_death_stats(diag)
    for k, v in bd.items():
        feats[f"{prefix}{k}"] = float(v)

    dd = _diag_distance_stats(diag)
    for k, v in dd.items():
        feats[f"{prefix}{k}"] = float(v)

    cxy = _centroid_xy(diag)
    feats[f"{prefix}centroid_x"] = float(cxy["centroid_x"])
    feats[f"{prefix}centroid_y"] = float(cxy["centroid_y"])

    q = _lifetimes_quantiles(diag)
    for k, v in q.items():
        feats[f"{prefix}{k}"] = float(v)

    tail80 = _tail_share_q(diag, 0.80)
    tail90 = _tail_share_q(diag, 0.90)
    tail95 = _tail_share_q(diag, 0.95)

    feats[f"{prefix}tail_share_q80"] = float(tail80)
    feats[f"{prefix}tail_share_q90"] = float(tail90)
    feats[f"{prefix}tail_share_q95"] = float(tail95)
    feats[f"{prefix}tail_curvature_80_90"] = float(tail90 - tail80)

    L = _lifetimes(diag)
    feats[f"{prefix}gini"] = float(_gini_from_lifetimes(L))

    cc = _carlsson_coordinates(diag)
    feats[f"{prefix}Carlsson_f1"] = float(cc["f1"])
    feats[f"{prefix}Carlsson_f2"] = float(cc["f2"])
    feats[f"{prefix}Carlsson_f3"] = float(cc["f3"])
    feats[f"{prefix}Carlsson_f4"] = float(cc["f4"])
    feats[f"{prefix}Carlsson_f5"] = float(cc["f5"])

    if prefix == "H0_":
        A = _auc_tri_max(diag)
        feats["H0_ratio_auc_L1_to_sum"] = _safe_div(A, Ls["sum"])
        feats["H0_ratio_auc_to_max"]    = _safe_div(A, Ls["max"])
        feats["H0_ratio_auc_to_l2"]     = _safe_div(A, Ls["L2"])
        feats["H0_bottleneck"]          = float(Ls["max"])
        feats["H0_sum_centroid"]        = float(_sum_centroid_radial(diag))
        feats["PETE_p1.6_q0.5"]         = float(_pete(diag, p=1.6, q=0.5))
        feats["H0_energy_concentration"]= _safe_div(Ls["L2"], Ls["sum"])
        feats["H0_dominance_share"]     = _safe_div(Ls["Linf"], Ls["sum"])
        feats["H0_tail_curvature_80_90"]= float(tail90 - tail80)
        feats["H0_centroid_to_energy"]  = _safe_div(feats["H0_sum_centroid"], Ls["L2"])
        feats["H0_gini"]                = float(feats["H0_gini"])
    return feats

def compute_cross_dim_features(feats_H0, feats_H1):
    out = {}
    def g(d, k): return float(d.get(k, 0.0))
    out["H1_to_H0_betti_ratio"]   = _safe_div(g(feats_H1, "H1_betti"), g(feats_H0, "H0_betti"))
    out["H1_to_H0_entropy_ratio"] = _safe_div(g(feats_H1, "H1_persistence_entropy"), g(feats_H0, "H0_persistence_entropy"))
    return out

# ==========================================================
# 3. ROBUST FEATURES LIST (77)
# ==========================================================
FEATURE_NAMES = [
    "H0_Carlsson_f1","H0_Carlsson_f3","H0_Carlsson_f5",
    "H0_L1_lifetime","H0_L1_norm","H0_L2_lifetime","H0_L2_norm",
    "H0_Linf_lifetime","H0_Linf_norm","H0_bottleneck","H0_centroid_to_energy",
    "H0_centroid_y","H0_dominance_share","H0_energy_concentration","H0_gini",
    "H0_max_diag_dist","H0_max_lifetime","H0_mean_death","H0_mean_diag_dist",
    "H0_mean_lifetime","H0_median_lifetime","H0_min_lifetime","H0_persistence_entropy",
    "H0_q50","H0_q75","H0_q90","H0_q95","H0_q99","H0_ratio_auc_L1_to_sum",
    "H0_ratio_auc_to_l2","H0_ratio_auc_to_max","H0_std_death","H0_std_lifetime",
    "H0_sum_centroid","H0_sum_diag_dist","H0_sum_lifetime","H0_tail_curvature_80_90",
    "H0_tail_share_q80","H0_tail_share_q90","H0_tail_share_q95",
    "H1_Carlsson_f1","H1_Carlsson_f2","H1_Carlsson_f3",
    "H1_L1_lifetime","H1_L1_norm","H1_L2_lifetime","H1_L2_norm",
    "H1_Linf_lifetime","H1_Linf_norm","H1_betti","H1_count_lifetime",
    "H1_dominance_share","H1_energy_concentration","H1_gini",
    "H1_max_diag_dist","H1_max_lifetime","H1_mean_diag_dist","H1_mean_lifetime",
    "H1_median_lifetime","H1_min_lifetime","H1_persistence_entropy",
    "H1_q50","H1_q75","H1_q90","H1_q95","H1_q99",
    "H1_std_birth","H1_std_death","H1_std_lifetime",
    "H1_sum_diag_dist","H1_sum_lifetime",
    "H1_tail_share_q80","H1_tail_share_q90","H1_tail_share_q95",
    "H1_to_H0_betti_ratio","H1_to_H0_entropy_ratio",
    "PETE_p1.6_q0.5"
]

# ==========================================================
# 4. MAIN: per file -> compute all features -> vote -> top-5 = 1 else 0
# ==========================================================
def run():
    files = glob.glob(os.path.join(INPUT_DIR, "**", "*.csv"), recursive=True)
    print(f"Found {len(files)} data files in '{INPUT_DIR}'")

    # Voting config
    K_PER_FEATURE = 3   # each feature votes for top-2 indices
    TOP_FINAL     = 10  # final anomalies = top-5 voted indices

    for filepath in files:
        if ".ipynb_checkpoints" in filepath:
            continue

        try:
            df = pd.read_csv(filepath)
            df.columns = [c.strip().lower() for c in df.columns]
            if "value" not in df.columns or "timestamp" not in df.columns:
                continue

            vals = pd.to_numeric(df["value"], errors="coerce").astype(float).to_numpy()
            n = len(vals)

            # ---- Build features for all windows ----
            rows = []
            for i in range(WINDOW_SIZE - 1, n):
                w = vals[i - WINDOW_SIZE + 1 : i + 1]
                try:
                    emb = takens_embed(w, TAU, DIMENSION)
                    dgms = ripser(emb, maxdim=MAXDIM)["dgms"]
                except Exception:
                    dgms = [np.empty((0, 2)), np.empty((0, 2))]

                D0 = dgms[0] if len(dgms) > 0 else np.empty((0, 2))
                D1 = dgms[1] if (MAXDIM >= 1 and len(dgms) > 1) else np.empty((0, 2))

                feats_H0 = compute_features_for_diag(D0, "H0_")
                feats_H1 = compute_features_for_diag(D1, "H1_")
                cross    = compute_cross_dim_features(feats_H0, feats_H1)

                merged = {}
                merged.update(feats_H0)
                merged.update(feats_H1)
                merged.update(cross)
                merged["index"] = i
                rows.append(merged)

            feat_df = pd.DataFrame(rows)
            full = pd.DataFrame(index=np.arange(n))
            if not feat_df.empty:
                feat_df = feat_df.set_index("index")
                full = full.join(feat_df, how="left")

            full = full.replace([np.inf, -np.inf], np.nan).fillna(0.0)

            # ---- Majority voting across ALL features ----
            votes = np.zeros(n, dtype=int)

            for feat_name in FEATURE_NAMES:
                series = pd.to_numeric(full.get(feat_name, 0.0), errors="coerce").astype(float).to_numpy()
                series = np.nan_to_num(series, nan=0.0, posinf=0.0, neginf=0.0)

                vead_scores = _vead_series(series, kv=KV, ka=KA, mode=MODE)

                mx = float(np.max(vead_scores)) if len(vead_scores) else 0.0
                if (not np.isfinite(mx)) or mx <= 0:
                    continue

                scores01 = np.clip(vead_scores / mx, 0.0, 1.0)
                if np.max(scores01) <= 0:
                    continue

                k_eff = min(K_PER_FEATURE, n)
                topk_idx = np.argpartition(scores01, -k_eff)[-k_eff:]

                # deterministic ordering (by score desc, then index asc)
                topk_idx = topk_idx[np.lexsort((topk_idx, -scores01[topk_idx]))]

                votes[topk_idx] += 1

            # ---- Take top-5 by vote count -> anomaly_score = 1 else 0 ----
            final_scores = np.zeros(n, dtype=float)
            if np.max(votes) > 0:
                top_final_eff = min(TOP_FINAL, n)
                order = np.lexsort((np.arange(n), -votes))  # votes desc, index asc
                chosen = order[:top_final_eff]
                final_scores[chosen] = 1.0

            # ---- Write output ----
            rel = os.path.relpath(filepath, INPUT_DIR)
            category = os.path.dirname(rel)
            base_name = os.path.basename(rel)

            out_dir = os.path.join(OUTPUT_DIR, category)
            os.makedirs(out_dir, exist_ok=True)
            out_name = f"{DETECTOR_NAME}_" + base_name
            out_path = os.path.join(out_dir, out_name)

            out_df = pd.DataFrame({
                "timestamp": df["timestamp"],
                "anomaly_score": final_scores
            })
            out_df.to_csv(out_path, index=False)
            print(f"-> Wrote: {out_path}")

        except Exception as e:
            print(f"!! Error processing {filepath}: {e}")
            continue

if __name__ == "__main__":
    run()
"""

with open("my_algo.py", "w") as f:
    f.write(tda_code)

print("✅ my_algo.py written.")

# ============================================================
# 3. RUN YOUR DETECTOR ON ALL NAB DATA
# ============================================================
print("--- 3. RUNNING TDA_VEAD_Method (ENSEMBLE) ON ALL DATASETS ---")
!python my_algo.py

# ============================================================
# 4. RUN NAB OPTIMIZE + SCORE FOR THIS DETECTOR
# ============================================================
print("--- 4. RUNNING NAB OPTIMIZE + SCORE ---")
!python run.py --optimize --score --detectors TDA_VEAD_Method --normalize


--- 1. CLEAN START ---
Cloning into 'NAB'...
remote: Enumerating objects: 7119, done.
remote: Counting objects: 100% (713/713), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 7119 (delta 601), reused 545 (delta 545), pack-reused 6406 (from 1)
Receiving objects: 100% (7119/7119), 86.73 MiB | 26.52 MiB/s, done.
Resolving deltas: 100% (5015/5015), done.
Updating files: 100% (1186/1186), done.
--- 2. WRITING TDA_VEAD_METHOD (ENSEMBLE VOTING) ---
✅ my_algo.py written.
--- 3. RUNNING TDA_VEAD_Method (ENSEMBLE) ON ALL DATASETS ---
Found 58 data files in 'data'
-> Wrote: results/TDA_VEAD_Method/realAdExchange/TDA_VEAD_Method_exchange-4_cpc_results.csv
-> Wrote: results/TDA_VEAD_Method/realAdExchange/TDA_VEAD_Method_exchange-2_cpc_results.csv
-> Wrote: results/TDA_VEAD_Method/realAdExchange/TDA_VEAD_Method_exchange-2_cpm_results.csv
-> Wrote: results/TDA_VEAD_Method/realAdExchange/TDA_VEAD_Method_exchange-3_cpm_results.csv
-> Wrote: results/TDA_VEAD_Method/realAdExchange